#Import

In [1]:
# if open from google colab, run this command
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/data-retail
# %cd /content/drive/MyDrive/Github/sales-prediction-gru/ipynb
# %cd /content/drive/Shareddrives/BUAT\ DIEGO/Data-retail-2015-2022-keluar/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/data-retail


In [2]:
import os, warnings, random

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import optimizers, Sequential, Model, callbacks
from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed 

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline 

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import joblib
from keras.models import load_model

pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 2017-2021

In [ ]:
df2017 = pd.read_csv('./data-2017-renew.csv', parse_dates=['tgl'])
df2018 = pd.read_csv('./data-2018-renew.csv', parse_dates=['tgl'])
df2019 = pd.read_csv('./data-2019-renew.csv', parse_dates=['tgl'])
df2020 = pd.read_csv('./data-2020-renew.csv', parse_dates=['tgl'])
df2021 = pd.read_csv('./data-2021-renew.csv', parse_dates=['tgl'])

In [ ]:
# drop unnecessary column
df2017 = df2017.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2018 = df2018.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2019 = df2019.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2020 = df2020.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2021 = df2021.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)

In [ ]:
# concat 5 dataframe to single df
df = pd.concat([df2017, df2018, df2019, df2020, df2021]).reset_index(drop=True)

In [ ]:
# remove strange value in each rows from barcode column
df['barcode'] = df['barcode'].str.split(' ').str[0]

In [ ]:
df.head(10)

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori
0,2017-10-08 10:21:56.667,SHINZUI SOAP REGULAR75GR*144,PCS,2.00,8992946511790,CU00000001,10301,Bath Care (1),103.00,Healt & Beauty Care (3)
1,2017-06-10 15:14:28.343,KOL PUTIH/KG,KG,0.19,00172677,CU00000001,20607,Vegetable (7),206.00,Fresh Food (6)
2,2017-09-18 20:38:05.290,PASEO SMART TOILET WHT 6143*64,PCS,1.00,8993053211672,CU00000001,10507,Tissue & Cotton (7),105.00,Cleaning (5)
3,2017-02-03 19:58:25.060,POND'S WB NIGHT CREAM 20GR/48,PCS,1.00,8999999056872,CU00000001,10302,Cosmetic & Accessories (2),103.00,Healt & Beauty Care (3)
4,2017-05-27 15:44:34.500,INDOMIE RASA SOTO SPESIAL 75GR,PCS,2.00,089686910384,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1)
5,2017-12-03 17:01:00.420,TB L/XL UK 30,PCS,1.00,56103,CU00000001,10601,Home Applience (1,106.00,Homes (6)
6,2017-10-31 15:57:49.297,INDOMIE KALDU AYAM,PCS,2.00,089686010107,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1)
7,2017-03-10 14:03:49.983,ROYCO FDS BEEF 100GR,PCS,1.00,8999999192198,CU00000001,20117,Seasoning (17),201.00,Food (1)
8,2017-07-07 18:50:42.257,PEPSODENT SNSTV EXPERT WHT 100GR,PCS,1.00,8999999032890,CU00000001,10307,Oral Care (7),103.00,Healt & Beauty Care (3)
9,2017-03-02 20:26:00.757,MOLTO EDP BLACK PURPLE 22ML/216 (18),BDD,2.00,8999999057640,CU00000001,10506,Softener (6),105.00,Cleaning (5)


In [ ]:
# drop unnesecary value from dataset 
df = df[df['kodecustomer'].isin(['CU00000001'])].reset_index(drop=True)
df = df[~df['barcode'].isin(['56101','56102','56103','56104'])].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('JASA PENDINGIN')].reset_index(drop=True)
df = df[~df['kategori'].str.contains('Toys', na=False)].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('ZANDILAC')].reset_index(drop=True)

In [ ]:
# create new column named "month_year" to get number of month and year
df['month_year'] = df['tgl'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [ ]:
# transform the value to number of month from all rows
# 2017-2021
df['month_year'] = df['month_year'].map({'01-17':0,'02-17':1,'03-17':2,'04-17':3,'05-17':4,'06-17':5,'07-17':6,'08-17':7,'09-17':8,'10-17':9,'11-17':10,'12-17':11,
                                          '01-18':12,'02-18':13,'03-18':14,'04-18':15,'05-18':16,'06-18':17,'07-18':18,'08-18':19,'09-18':20,'10-18':21,'11-18':22,'12-18':23,
                                          '01-19':24,'02-19':25,'03-19':26,'04-19':27,'05-19':28,'06-19':29,'07-19':30,'08-19':31,'09-19':32,'10-19':33,'11-19':34,'12-19':35,
                                          '01-20':36,'02-20':37,'03-20':38,'04-20':39,'05-20':40,'06-20':41,'07-20':42,'08-20':43,'09-20':44,'10-20':45,'11-20':46,'12-20':47,
                                          '01-21':48,'02-21':49,'03-21':50,'04-21':51,'05-21':52,'06-21':53,'07-21':54,'08-21':55,'09-21':56,'10-21':57,'11-21':58,'12-21':59})

In [ ]:
display(df.info())
display(df.head())
# RangeIndex: 11533836 entries, 0 to 11533835 sebelum nambah daftar kategori

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11535921 entries, 0 to 11535920
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   tgl           datetime64[ns]
 1   namabarang    object        
 2   satuan        object        
 3   qty           float64       
 4   barcode       object        
 5   kodecustomer  object        
 6   kodejenis     object        
 7   jenis         object        
 8   kodekategori  float64       
 9   kategori      object        
 10  month_year    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 968.1+ MB


None

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori,month_year
0,2017-10-08 10:21:56.667,SHINZUI SOAP REGULAR75GR*144,PCS,2.00,8992946511790,CU00000001,10301,Bath Care (1),103.00,Healt & Beauty Care (3),9
1,2017-06-10 15:14:28.343,KOL PUTIH/KG,KG,0.19,00172677,CU00000001,20607,Vegetable (7),206.00,Fresh Food (6),5
2,2017-09-18 20:38:05.290,PASEO SMART TOILET WHT 6143*64,PCS,1.00,8993053211672,CU00000001,10507,Tissue & Cotton (7),105.00,Cleaning (5),8
3,2017-02-03 19:58:25.060,POND'S WB NIGHT CREAM 20GR/48,PCS,1.00,8999999056872,CU00000001,10302,Cosmetic & Accessories (2),103.00,Healt & Beauty Care (3),1
4,2017-05-27 15:44:34.500,INDOMIE RASA SOTO SPESIAL 75GR,PCS,2.00,089686910384,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1),4


In [ ]:
train_monthly = df[['tgl', 'month_year', 'kodejenis', 'barcode', 'qty']].copy()
train_monthly = train_monthly.sort_values('tgl').groupby(['month_year', 'kodejenis', 'barcode'], as_index=False)
train_monthly = train_monthly.agg({'qty':['sum']})
train_monthly.columns = ['month_year', 'kodejenis', 'barcode', 'qty_cnt']
train_monthly = train_monthly.query('qty_cnt > 0')

train_monthly['qty_cnt_month'] = train_monthly.sort_values('month_year').groupby(['barcode', 'kodejenis'])['qty_cnt'].shift(-1)

display(train_monthly.head())

,month_year,kodejenis,barcode,qty_cnt,qty_cnt_month
0,0,10101,4902430600859,2.00,NaN
1,0,10101,4902430601191,4.00,2.00
2,0,10101,4902430601238,4.00,1.00
3,0,10101,4902430601245,7.00,NaN
4,0,10101,4902430601320,1.00,1.00


In [ ]:
monthly_series = train_monthly.pivot_table(index=['barcode', 'kodejenis'], columns='month_year',values='qty_cnt', fill_value=0).reset_index()
monthly_series.head()

month_year,barcode,kodejenis,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,00100809,01011601,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00100809,20604,8.04,10.89,15.34,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55,19.50,15.83,13.72,19.72,20.78,19.90,12.00,11.78,14.68,14.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,00113731,02020101,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.60,3.46,4.43,4.75,6.09,4.07,5.42,4.27,4.89,3.76,3.62,2.89,3.61,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.95,0.75,0.78,0.45,1.72,0.32
3,00113731,20607,1.51,0.00,0.00,0.00,0.00,0.00,0.00,1.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,00125789,02010120,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.72,0.00,0.00,0.00,0.00,9.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
monthly_series_drop = monthly_series.drop(['kodejenis'], axis=1)

In [ ]:
summed_data = monthly_series_drop.groupby('barcode').agg('sum').reset_index(False)
summed_data.head()

month_year,barcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,00100809,8.04,10.89,15.34,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55,19.50,15.83,13.72,19.72,20.78,19.90,12.00,11.78,14.68,14.05,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00113731,1.51,0.00,0.00,0.00,0.00,0.00,0.00,1.49,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.60,3.46,4.43,4.75,6.09,4.07,5.42,4.27,4.89,3.76,3.62,2.89,3.61,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.95,0.75,0.78,0.45,1.72,0.32
2,00125789,0.00,0.00,0.00,0.00,6.52,0.00,30.61,1.15,0.00,0.00,1.27,24.00,0.00,0.00,0.00,0.84,0.67,23.76,7.98,21.34,0.00,18.12,0.00,1.40,0.72,0.00,0.00,0.00,0.00,9.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,00129213,54.89,38.54,15.94,11.55,0.00,0.00,0.00,0.00,1.40,0.00,34.27,20.00,8.27,1.42,0.00,0.00,0.00,0.00,0.63,0.00,0.00,5.38,31.02,22.49,7.61,15.97,10.74,6.98,10.05,0.00,0.00,0.00,0.00,0.00,17.51,0.00,79.86,212.27,5.13,0.60,2.79,8.18,0.00,0.00,0.00,4.83,7.70,1.50,38.39,60.46,1.95,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,00133142,0.00,4.19,6.89,0.00,0.00,0.00,0.00,3.46,0.00,0.00,0.00,0.00,0.00,0.00,10.13,24.48,2.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
first_month = 0
last_month = 59
serie_size = 12
data_series = []

for index, row in summed_data.iterrows():
    for month1 in range((last_month - (first_month + serie_size)) + 1):
        # serie = [row['barcode'], row['kodejenis']]
        serie = [row['barcode']]
        for month2 in range(serie_size + 1):
            serie.append(row[month1 + first_month + month2])
        data_series.append(serie)

columns = ['barcode']
[columns.append(i) for i in range(serie_size)]
columns.append('label')
data_series = pd.DataFrame(data_series, columns=columns)
data_series.head()

,barcode,0,1,2,3,4,5,6,7,8,9,10,11,label
0,00100809,8.04,10.89,15.34,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40
1,00100809,10.89,15.34,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55
2,00100809,15.34,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55,19.50
3,00100809,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55,19.50,15.83
4,00100809,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55,19.50,15.83,13.72


In [ ]:
data_series = data_series.drop(['barcode'], axis=1)
data_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,8.04,10.89,15.34,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40
1,10.89,15.34,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55
2,15.34,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55,19.50
3,14.70,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55,19.50,15.83
4,15.63,20.85,11.92,16.73,14.30,14.26,11.93,11.39,14.40,12.55,19.50,15.83,13.72


In [ ]:
outname = 'data_series_2017_2021.csv'
outdir = './data-train-unlimit'

if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

data_series.to_csv(fullname)

# 2018-2021

In [20]:
df2018 = pd.read_csv('./data-2018-renew.csv', parse_dates=['tgl'])
df2019 = pd.read_csv('./data-2019-renew.csv', parse_dates=['tgl'])
df2020 = pd.read_csv('./data-2020-renew.csv', parse_dates=['tgl'])
df2021 = pd.read_csv('./data-2021-renew.csv', parse_dates=['tgl'])

In [21]:
# drop unnecessary column
df2018 = df2018.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2019 = df2019.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2020 = df2020.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2021 = df2021.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)

In [22]:
# concat 5 dataframe to single df
df = pd.concat([df2018, df2019, df2020, df2021]).reset_index(drop=True)

In [23]:
# remove strange value in each rows from barcode column
df['barcode'] = df['barcode'].str.split(' ').str[0]

In [24]:
df.head(10)

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori
0,2018-04-18 19:18:51.523,TB S/M UK 25,PCS,1.00,56102,CU00000001,10601,Home Applience (1,106.00,Homes (6)
1,2018-11-19 18:19:38.213,INDOMIE GORENG KRIUK PEDAS,PCS,1.00,089686059779,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1)
2,2018-10-12 17:31:53.357,SARIMI RASA AYAM (RRABG),PCS,16.00,089686017700,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1)
3,2018-07-15 18:36:40.493,VAPE MAT ULTRA 60'S/45'S,PCS,1.00,8992857050135,CU00000001,10505,Pes Control (5),105.00,Cleaning (5)
4,2018-12-08 21:33:23.330,PENGHAPUS STEADLER 526 B40*40,PCS,1.00,4007817524206,CU00000001,10903,Stationery (3),109.00,Stationery (9)
5,2018-03-25 20:22:25.703,CHARM L&W BREATHABLE NP 40P,PCS,1.00,8993189272066,CU00000001,10306,Napkin (6),103.00,Healt & Beauty Care (3)
6,2018-07-04 12:36:03.487,ROK SUN SHINE 1208,PCS,1.00,85738,CU00000001,11303,Kids Wear (3),113.00,Convection (13)
7,2018-06-21 18:57:34.150,SUNLIGHT LIME REF 240ML/24,PCS,1.00,8999999059781,CU00000001,10503,Home & Leater Care (3),105.00,Cleaning (5)
8,2018-10-31 20:53:14.920,BIORE BF LOVELY SAKURA POUCH 250ML*24,PCS,1.00,8992727006170,CU00000001,10301,Bath Care (1),103.00,Healt & Beauty Care (3)
9,2018-12-11 13:47:03.247,"PARUTAN MENARA TG 18"" BIASA",PCS,1.00,13238,CU00000001,10602,Kitchen (2),106.00,Homes (6)


In [25]:
# drop unnesecary value from dataset 
df = df[df['kodecustomer'].isin(['CU00000001'])].reset_index(drop=True)
# df = df[(df['namabarang'].str.contains('TB')) & (df['kategori'] == 'Homes (6)') & (df['barcode'].isin(['56101','56102','56103','56104']))]
df = df[~df['barcode'].isin(['56101','56102','56103','56104'])].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('JASA PENDINGIN')].reset_index(drop=True)
df = df[~df['kategori'].str.contains('Toys', na=False)].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('ZANDILAC')].reset_index(drop=True)
# df = df.query('qty > 0 & qty <= 200').sort_values(by="qty", ascending=False).reset_index(drop=True)
# df = df.query('qty > 0 & qty <= 300').sort_values(by="qty", ascending=False).reset_index(drop=True)
# df = df.query('qty > 0 & qty <= 400').sort_values(by="qty", ascending=False).reset_index(drop=True)
df = df.query('qty > 0 & qty <= 500').sort_values(by="qty", ascending=False).reset_index(drop=True)

In [26]:
# create new column named "month_year" to get number of month and year
df['month_year'] = df['tgl'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [27]:
df.head()

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori,month_year
0,2019-08-01 11:32:02.560,TAS BELANJA OXIUM 37*100PCS,PCS,500.00,99956,CU00000001,04031202,Packaging,40312.00,Perlengkapan Acara (Event Equipment),08-19
1,2019-08-01 11:31:43.517,TAS BELANJA OXIUM 30*100PCS,PCS,500.00,99955,CU00000001,04031202,Packaging,40312.00,Perlengkapan Acara (Event Equipment),08-19
2,2021-12-17 10:17:15.020,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209.00,Minuman Penyegar (Refreshment),12-21
3,2021-12-17 10:16:03.893,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209.00,Minuman Penyegar (Refreshment),12-21
4,2021-12-17 10:13:33.797,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209.00,Minuman Penyegar (Refreshment),12-21


In [28]:
# transform the value to number of month from all rows
2018-2021
df['month_year'] = df['month_year'].map({'01-18':0,'02-18':1,'03-18':2,'04-18':3,'05-18':4,'06-18':5,'07-18':6,'08-18':7,'09-18':8,'10-18':9,'11-18':10,'12-18':11,
                                          '01-19':12,'02-19':13,'03-19':14,'04-19':15,'05-19':16,'06-19':17,'07-19':18,'08-19':19,'09-19':20,'10-19':21,'11-19':22,'12-19':23,
                                          '01-20':24,'02-20':25,'03-20':26,'04-20':27,'05-20':28,'06-20':29,'07-20':30,'08-20':31,'09-20':32,'10-20':33,'11-20':34,'12-20':35,
                                          '01-21':36,'02-21':37,'03-21':38,'04-21':39,'05-21':40,'06-21':41,'07-21':42,'08-21':43,'09-21':44,'10-21':45,'11-21':46,'12-21':47})

In [29]:
display(df.info())
display(df.head())
# RangeIndex: 8988342 entries, 0 to 8988341 = limit 200
# RangeIndex: 8988412 entries, 0 to 8988411 = limit 300
# RangeIndex: 8988432 entries, 0 to 8988431 = limit 400
# RangeIndex: 8988461 entries, 0 to 8988460 = limit 500

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8988461 entries, 0 to 8988460
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   tgl           datetime64[ns]
 1   namabarang    object        
 2   satuan        object        
 3   qty           float64       
 4   barcode       object        
 5   kodecustomer  object        
 6   kodejenis     object        
 7   jenis         object        
 8   kodekategori  float64       
 9   kategori      object        
 10  month_year    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 754.3+ MB


None

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori,month_year
0,2019-08-01 11:32:02.560,TAS BELANJA OXIUM 37*100PCS,PCS,500.00,99956,CU00000001,04031202,Packaging,40312.00,Perlengkapan Acara (Event Equipment),19
1,2019-08-01 11:31:43.517,TAS BELANJA OXIUM 30*100PCS,PCS,500.00,99955,CU00000001,04031202,Packaging,40312.00,Perlengkapan Acara (Event Equipment),19
2,2021-12-17 10:17:15.020,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209.00,Minuman Penyegar (Refreshment),47
3,2021-12-17 10:16:03.893,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209.00,Minuman Penyegar (Refreshment),47
4,2021-12-17 10:13:33.797,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209.00,Minuman Penyegar (Refreshment),47


In [30]:
train_monthly = df[['tgl', 'month_year', 'kodejenis', 'barcode', 'qty']].copy()
train_monthly = train_monthly.sort_values('tgl').groupby(['month_year', 'kodejenis', 'barcode'], as_index=False)
train_monthly = train_monthly.agg({'qty':['sum']})
train_monthly.columns = ['month_year', 'kodejenis', 'barcode', 'qty_cnt']
# train_monthly = train_monthly.query('qty_cnt > 0 & qty_cnt <= 200')
# train_monthly = train_monthly.query('qty_cnt > 0 & qty_cnt <= 300')
# train_monthly = train_monthly.query('qty_cnt > 0 & qty_cnt <= 400')
train_monthly = train_monthly.query('qty_cnt > 0 & qty_cnt <= 500')

train_monthly['qty_cnt_month'] = train_monthly.sort_values('month_year').groupby(['barcode', 'kodejenis'])['qty_cnt'].shift(-1)

display(train_monthly.head())

,month_year,kodejenis,barcode,qty_cnt,qty_cnt_month
0,0,10101,089686960044,2.00,3.00
1,0,10101,089686960075,1.00,2.00
2,0,10101,089686960396,5.00,8.00
3,0,10101,4902430605144,4.00,7.00
4,0,10101,4902430610346,2.00,3.00


In [31]:
monthly_series = train_monthly.pivot_table(index=['barcode', 'kodejenis'], columns='month_year',values='qty_cnt', fill_value=0).reset_index()
monthly_series.head()

month_year,barcode,kodejenis,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,00100809,01011601,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00100809,20604,14.40,12.55,19.50,15.83,13.72,19.72,20.78,19.90,12.00,11.78,14.68,14.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,00113731,02020101,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.60,3.46,4.43,4.75,6.09,4.07,5.42,4.27,4.89,3.76,3.62,2.89,3.61,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.95,0.75,0.78,0.45,1.72,0.32
3,00125789,02010120,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.72,0.00,0.00,0.00,0.00,9.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,00125789,20602,0.00,0.00,0.00,0.84,0.67,23.76,7.98,21.34,0.00,18.12,0.00,1.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [32]:
monthly_series_drop = monthly_series.drop(['kodejenis'], axis=1)

In [33]:
summed_data = monthly_series_drop.groupby('barcode').agg('sum').reset_index(False)
summed_data.head()

month_year,barcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,00100809,14.40,12.55,19.50,15.83,13.72,19.72,20.78,19.90,12.00,11.78,14.68,14.05,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00113731,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.60,3.46,4.43,4.75,6.09,4.07,5.42,4.27,4.89,3.76,3.62,2.89,3.61,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.95,0.75,0.78,0.45,1.72,0.32
2,00125789,0.00,0.00,0.00,0.84,0.67,23.76,7.98,21.34,0.00,18.12,0.00,1.40,0.72,0.00,0.00,0.00,0.00,9.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,00129213,8.27,1.42,0.00,0.00,0.00,0.00,0.63,0.00,0.00,5.38,31.02,22.49,7.61,15.97,10.74,6.98,10.05,0.00,0.00,0.00,0.00,0.00,17.51,0.00,80.89,212.27,5.13,0.60,2.79,8.18,0.00,0.00,0.00,4.83,7.70,1.50,38.39,60.46,1.95,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,00133142,0.00,0.00,10.13,24.48,2.14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [34]:
first_month = 0
last_month = 47
serie_size = 12
data_series = []

for index, row in summed_data.iterrows():
    for month1 in range((last_month - (first_month + serie_size)) + 1):
        # serie = [row['barcode'], row['kodejenis']]
        serie = [row['barcode']]
        for month2 in range(serie_size + 1):
            serie.append(row[month1 + first_month + month2])
        data_series.append(serie)

columns = ['barcode']
[columns.append(i) for i in range(serie_size)]
columns.append('label')
data_series = pd.DataFrame(data_series, columns=columns)
data_series.head()

,barcode,0,1,2,3,4,5,6,7,8,9,10,11,label
0,00100809,14.40,12.55,19.50,15.83,13.72,19.72,20.78,19.90,12.00,11.78,14.68,14.05,18.69
1,00100809,12.55,19.50,15.83,13.72,19.72,20.78,19.90,12.00,11.78,14.68,14.05,18.69,15.35
2,00100809,19.50,15.83,13.72,19.72,20.78,19.90,12.00,11.78,14.68,14.05,18.69,15.35,20.38
3,00100809,15.83,13.72,19.72,20.78,19.90,12.00,11.78,14.68,14.05,18.69,15.35,20.38,13.80
4,00100809,13.72,19.72,20.78,19.90,12.00,11.78,14.68,14.05,18.69,15.35,20.38,13.80,7.03


In [ ]:
data_series = data_series.drop(['barcode'], axis=1)
data_series.head()

In [ ]:
# outname = 'data_series_2018_2021.csv'
# outname = 'data_series_2018_2021_limit200.csv'
# outname = 'data_series_2018_2021_limit300.csv'
# outname = 'data_series_2018_2021_limit400.csv'
outname = 'data_series_2018_2021_limit500.csv'

outdir = './data-train-unlimit'

if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

data_series.to_csv(fullname)

# 2019-2021

In [ ]:
df2019 = pd.read_csv('./data-2019-renew.csv', parse_dates=['tgl'])
df2020 = pd.read_csv('./data-2020-renew.csv', parse_dates=['tgl'])
df2021 = pd.read_csv('./data-2021-renew.csv', parse_dates=['tgl'])

In [ ]:
# drop unnecessary column
df2019 = df2019.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2020 = df2020.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2021 = df2021.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)

In [ ]:
# concat 5 dataframe to single df
df = pd.concat([df2019, df2020, df2021]).reset_index(drop=True)

In [ ]:
# remove strange value in each rows from barcode column
df['barcode'] = df['barcode'].str.split(' ').str[0]

In [ ]:
df.head(10)

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori
0,2019-09-24 11:19:44.360,VIT 600ML*24,PCS,1.00,8992752011408,CU00000001,01020403,Air Mineral Kemasan Botol,10204,Air (Water)
1,2019-09-08 16:09:13.810,BEAR BRAND 195GR*30,KLG,3.00,8992696404441,CU00000001,01020109,Susu Cair (RTD),10201,Susu (Milk)
2,2019-01-22 18:53:17.497,INDOMIE GORENG IGA PENYET,PCS,1.00,089686043686,CU00000001,01011202,Mie Instant,10112,Mie (Noodle)
3,2019-01-28 18:08:18.700,GARUDA ATOM MANIS AJTM130GR*60,BKS,3.00,8992775204009,CU00000001,01011403,Kacang Modern,10114,Makanan Ringan (Snack)
4,2019-02-24 20:38:04.383,INDOMIE MIE GORENG KHAS ACEH 90GR*40,PCS,1.00,089686043204,CU00000001,01011202,Mie Instant,10112,Mie (Noodle)
5,2019-09-30 18:50:43.320,SAORI SAUS TIRAM 23GR*10,PCS,2.00,8992770094117,CU00000001,01011104,Bumbu Saus,10111,Bumbu Dapur (Seasoning)
6,2019-10-26 19:46:10.427,DETTOL DAS ENERGIZE 110GR* 3S,BDD,1.00,8993560024901,CU00000001,03030601,Sabun Mandi Batang Antiseptic,30306,Perawatan Tubuh (Body Care)
7,2019-09-12 18:15:20.927,SUNLIGHT LIME REF 220ML/24,PCS,1.00,8999999059781,CU00000001,03040201,Sabun Cair,30402,Perawatan Dapur (Kitchen Care)
8,2019-06-21 17:47:50.450,ENTRASOL GOLD VANILA 185GR,PCS,1.00,8992802618731,CU00000001,01020105,Susu Kalsium,10201,Susu (Milk)
9,2019-09-01 18:58:05.730,CHARM LONG&WIDE EXT.DRY NP 40P,PCS,1.00,8993189270765,CU00000001,03031001,Panty Liner,30310,Pembalut Wanita (Sanitary Napkins)


In [ ]:
# drop unnesecary value from dataset 
df = df[df['kodecustomer'].isin(['CU00000001'])].reset_index(drop=True)
df = df[~df['barcode'].isin(['56101','56102','56103','56104'])].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('JASA PENDINGIN')].reset_index(drop=True)
df = df[~df['kategori'].str.contains('Toys', na=False)].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('ZANDILAC')].reset_index(drop=True)

In [ ]:
# create new column named "month_year" to get number of month and year
df['month_year'] = df['tgl'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [ ]:
# transform the value to number of month from all rows
# 2019-2021
df['month_year'] = df['month_year'].map({'01-19':0,'02-19':1,'03-19':2,'04-19':3,'05-19':4,'06-19':5,'07-19':6,'08-19':7,'09-19':8,'10-19':9,'11-19':10,'12-19':11,
                                          '01-20':12,'02-20':13,'03-20':14,'04-20':15,'05-20':16,'06-20':17,'07-20':18,'08-20':19,'09-20':20,'10-20':21,'11-20':22,'12-20':23,
                                          '01-21':24,'02-21':25,'03-21':26,'04-21':27,'05-21':28,'06-21':29,'07-21':30,'08-21':31,'09-21':32,'10-21':33,'11-21':34,'12-21':35})

In [ ]:
display(df.info())
display(df.head())
# RangeIndex: 11533836 entries, 0 to 11533835 sebelum nambah daftar kategori

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6339043 entries, 0 to 6339042
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   tgl           datetime64[ns]
 1   namabarang    object        
 2   satuan        object        
 3   qty           float64       
 4   barcode       object        
 5   kodecustomer  object        
 6   kodejenis     object        
 7   jenis         object        
 8   kodekategori  int64         
 9   kategori      object        
 10  month_year    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 532.0+ MB


None

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori,month_year
0,2019-09-24 11:19:44.360,VIT 600ML*24,PCS,1.00,8992752011408,CU00000001,01020403,Air Mineral Kemasan Botol,10204,Air (Water),8
1,2019-09-08 16:09:13.810,BEAR BRAND 195GR*30,KLG,3.00,8992696404441,CU00000001,01020109,Susu Cair (RTD),10201,Susu (Milk),8
2,2019-01-22 18:53:17.497,INDOMIE GORENG IGA PENYET,PCS,1.00,089686043686,CU00000001,01011202,Mie Instant,10112,Mie (Noodle),0
3,2019-01-28 18:08:18.700,GARUDA ATOM MANIS AJTM130GR*60,BKS,3.00,8992775204009,CU00000001,01011403,Kacang Modern,10114,Makanan Ringan (Snack),0
4,2019-02-24 20:38:04.383,INDOMIE MIE GORENG KHAS ACEH 90GR*40,PCS,1.00,089686043204,CU00000001,01011202,Mie Instant,10112,Mie (Noodle),1


In [ ]:
train_monthly = df[['tgl', 'month_year', 'kodejenis', 'barcode', 'qty']].copy()
train_monthly = train_monthly.sort_values('tgl').groupby(['month_year', 'kodejenis', 'barcode'], as_index=False)
train_monthly = train_monthly.agg({'qty':['sum']})
train_monthly.columns = ['month_year', 'kodejenis', 'barcode', 'qty_cnt']
train_monthly = train_monthly.query('qty_cnt > 0')

train_monthly['qty_cnt_month'] = train_monthly.sort_values('month_year').groupby(['barcode', 'kodejenis'])['qty_cnt'].shift(-1)

display(train_monthly.head())

,month_year,kodejenis,barcode,qty_cnt,qty_cnt_month
0,0,01010101,089686640007,33.00,34.00
1,0,01010101,089686640014,29.00,34.00
2,0,01010101,089686646047,9.00,16.00
3,0,01010101,089686646061,3.00,10.00
4,0,01010101,089686646450,1.00,2.00


In [ ]:
monthly_series = train_monthly.pivot_table(index=['barcode', 'kodejenis'], columns='month_year',values='qty_cnt', fill_value=0).reset_index()
monthly_series.head()

month_year,barcode,kodejenis,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,00100809,01011601,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00113731,02020101,0.00,0.00,2.60,3.46,4.43,4.75,6.09,4.07,5.42,4.27,4.89,3.76,3.62,2.89,3.61,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.95,0.75,0.78,0.45,1.72,0.32
2,00125789,02010120,0.72,0.00,0.00,0.00,0.00,9.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,00129213,02010113,7.61,15.97,10.74,6.98,10.05,0.00,0.00,0.00,0.00,0.00,17.51,0.00,79.86,212.27,5.13,0.60,2.79,8.18,0.00,0.00,0.00,4.83,7.70,1.50,38.39,60.46,1.95,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,00133173,02010137,481.56,394.04,538.46,305.12,249.99,67.82,179.99,130.33,79.80,225.32,356.21,501.47,486.84,570.40,472.06,388.09,224.17,200.57,181.96,115.26,880.17,199.02,163.49,160.73,178.80,140.58,197.99,985.26,165.40,66.73,53.37,47.26,224.66,189.91,884.29,180.94


In [ ]:
monthly_series_drop = monthly_series.drop(['kodejenis'], axis=1)

In [ ]:
summed_data = monthly_series_drop.groupby('barcode').agg('sum').reset_index(False)
summed_data.head()

month_year,barcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,00100809,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00113731,0.00,0.00,2.60,3.46,4.43,4.75,6.09,4.07,5.42,4.27,4.89,3.76,3.62,2.89,3.61,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.95,0.75,0.78,0.45,1.72,0.32
2,00125789,0.72,0.00,0.00,0.00,0.00,9.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,00129213,7.61,15.97,10.74,6.98,10.05,0.00,0.00,0.00,0.00,0.00,17.51,0.00,79.86,212.27,5.13,0.60,2.79,8.18,0.00,0.00,0.00,4.83,7.70,1.50,38.39,60.46,1.95,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,00133173,481.56,394.04,538.46,305.12,249.99,67.82,179.99,130.33,79.80,225.32,356.21,501.47,486.84,570.40,472.06,388.09,224.17,200.57,181.96,115.26,880.17,199.02,163.49,160.73,178.80,140.58,197.99,985.26,165.40,66.73,53.37,47.26,224.66,189.91,884.29,180.94


In [ ]:
first_month = 0
last_month = 35
serie_size = 12
data_series = []

for index, row in summed_data.iterrows():
    for month1 in range((last_month - (first_month + serie_size)) + 1):
        # serie = [row['barcode'], row['kodejenis']]
        serie = [row['barcode']]
        for month2 in range(serie_size + 1):
            serie.append(row[month1 + first_month + month2])
        data_series.append(serie)

columns = ['barcode']
[columns.append(i) for i in range(serie_size)]
columns.append('label')
data_series = pd.DataFrame(data_series, columns=columns)
data_series.head()

,barcode,0,1,2,3,4,5,6,7,8,9,10,11,label
0,00100809,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75
1,00100809,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78
2,00100809,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13
3,00100809,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93
4,00100809,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57


In [ ]:
data_series = data_series.drop(['barcode'], axis=1)
data_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75
1,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78
2,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13
3,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93
4,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57


In [ ]:
outname = 'data_series_2019_2021.csv'
outdir = './data-train-unlimit'

if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

data_series.to_csv(fullname)

# 2020-2021

In [36]:
df2020 = pd.read_csv('./data-2020-renew.csv', parse_dates=['tgl'])
df2021 = pd.read_csv('./data-2021-renew.csv', parse_dates=['tgl'])

In [37]:
# drop unnecessary column
df2020 = df2020.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2021 = df2021.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)

In [38]:
# concat 5 dataframe to single df
df = pd.concat([df2020, df2021]).reset_index(drop=True)

In [39]:
# remove strange value in each rows from barcode column
df['barcode'] = df['barcode'].str.split(' ').str[0]

In [40]:
df.head()

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori
0,2020-04-03 19:27:36.123,MASAKO PENYEDAP RS.SAPI/PAK,PAK,1.00,8992770033208,CU00000001,01011108,Bumbu Tabur,10111,Bumbu Dapur (Seasoning)
1,2020-10-21 16:47:57.980,RINSO 770GR ANTINODA,PCS,1.00,8999999401238,CU00000001,03040101,Detergent Bubuk,30401,Perawatan Pakaian (Clothes Care)
2,2020-04-06 17:46:52.900,CAREFREE HLT.FRESH SPR DRY 40S,PCS,1.00,8850007331803,CU00000001,03031001,Panty Liner,30310,Pembalut Wanita (Sanitary Napkins)
3,2020-07-31 19:15:58.547,ELLLIPS VIT HR NUTRI CLR BLS,PCS,2.00,8993417200038,CU00000001,03030105,Vitamin Rambut,30301,Perawatan Rambut (Hair Care)
4,2020-05-22 13:14:43.407,KUSUKA KRIPIK RUMPUT LAUT 60GR*30,PCS,1.00,653314504513,CU00000001,01011401,Snack Modern,10114,Makanan Ringan (Snack)


In [41]:
# drop unnesecary value from dataset 
df = df[df['kodecustomer'].isin(['CU00000001'])].reset_index(drop=True)
df = df[~df['barcode'].isin(['56101','56102','56103','56104'])].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('JASA PENDINGIN')].reset_index(drop=True)
df = df[~df['kategori'].str.contains('Toys', na=False)].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('ZANDILAC')].reset_index(drop=True)
# df = df.query('qty > 0 & qty <= 200').sort_values(by="qty", ascending=False).reset_index(drop=True)
# df = df.query('qty > 0 & qty <= 300').sort_values(by="qty", ascending=False).reset_index(drop=True)
# df = df.query('qty > 0 & qty <= 400').sort_values(by="qty", ascending=False).reset_index(drop=True)
df = df.query('qty > 0 & qty <= 500').sort_values(by="qty", ascending=False).reset_index(drop=True)

In [42]:
# create new column named "month_year" to get number of month and year
df['month_year'] = df['tgl'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [43]:
# transform the value to number of month from all rows
# 2020-2021
df['month_year'] = df['month_year'].map({'01-20':0,'02-20':1,'03-20':2,'04-20':3,'05-20':4,'06-20':5,'07-20':6,'08-20':7,'09-20':8,'10-20':9,'11-20':10,'12-20':11,
                                          '01-21':12,'02-21':13,'03-21':14,'04-21':15,'05-21':16,'06-21':17,'07-21':18,'08-21':19,'09-21':20,'10-21':21,'11-21':22,'12-21':23})

In [44]:
display(df.info())
display(df.head())
# RangeIndex: 3771076 entries, 0 to 3771075 = limit 300
# RangeIndex: 3771085 entries, 0 to 3771084 = limit 400
# RangeIndex: 3771110 entries, 0 to 3771109 = limit 500

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3771110 entries, 0 to 3771109
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   tgl           datetime64[ns]
 1   namabarang    object        
 2   satuan        object        
 3   qty           float64       
 4   barcode       object        
 5   kodecustomer  object        
 6   kodejenis     object        
 7   jenis         object        
 8   kodekategori  int64         
 9   kategori      object        
 10  month_year    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 316.5+ MB


None

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori,month_year
0,2021-12-17 10:08:53.667,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209,Minuman Penyegar (Refreshment),23
1,2021-12-08 12:23:14.040,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209,Minuman Penyegar (Refreshment),23
2,2021-12-17 10:06:03.310,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209,Minuman Penyegar (Refreshment),23
3,2021-12-08 12:22:32.280,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209,Minuman Penyegar (Refreshment),23
4,2021-12-17 10:17:15.020,ADEM SARI CHINGKU 350ML BTL,BTL,480.00,8992772585026,CU00000001,01020902,Minuman Penyegar Cair (RTD),10209,Minuman Penyegar (Refreshment),23


In [45]:
train_monthly = df[['tgl', 'month_year', 'kodejenis', 'barcode', 'qty']].copy()
train_monthly = train_monthly.sort_values('tgl').groupby(['month_year', 'kodejenis', 'barcode'], as_index=False)
train_monthly = train_monthly.agg({'qty':['sum']})
train_monthly.columns = ['month_year', 'kodejenis', 'barcode', 'qty_cnt']
# train_monthly = train_monthly.query('qty_cnt > 0 & qty_cnt <= 200')
# train_monthly = train_monthly.query('qty_cnt > 0 & qty_cnt <= 300')
# train_monthly = train_monthly.query('qty_cnt > 0 & qty_cnt <= 400')
train_monthly = train_monthly.query('qty_cnt > 0 & qty_cnt <= 500')

train_monthly['qty_cnt_month'] = train_monthly.sort_values('month_year').groupby(['barcode', 'kodejenis'])['qty_cnt'].shift(-1)

display(train_monthly.head())

,month_year,kodejenis,barcode,qty_cnt,qty_cnt_month
0,0,01010101,089686640007,30.00,24.00
1,0,01010101,089686640014,30.00,34.00
2,0,01010101,089686640045,12.00,12.00
3,0,01010101,089686646047,7.00,3.00
4,0,01010101,089686646061,5.00,6.00


In [46]:
monthly_series = train_monthly.pivot_table(index=['barcode', 'kodejenis'], columns='month_year',values='qty_cnt', fill_value=0).reset_index()
monthly_series.head()

month_year,barcode,kodejenis,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,00100809,01011601,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00113731,02020101,3.62,2.89,3.61,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.95,0.75,0.78,0.45,1.72,0.32
2,00129213,02010113,80.89,212.27,5.13,0.60,2.79,8.18,0.00,0.00,0.00,4.83,7.70,1.50,38.39,60.46,1.95,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,00133173,02010137,486.84,0.00,472.06,388.09,224.17,200.57,181.96,115.26,0.00,199.02,163.49,160.73,178.80,140.58,197.99,0.00,165.40,66.73,53.37,47.26,224.66,189.91,0.00,180.94
4,00133210,02010113,0.00,0.00,1.00,2.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [47]:
monthly_series_drop = monthly_series.drop(['kodejenis'], axis=1)

In [48]:
summed_data = monthly_series_drop.groupby('barcode').agg('sum').reset_index(False)
summed_data.head()

month_year,barcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,00100809,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00113731,3.62,2.89,3.61,0.76,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.95,0.75,0.78,0.45,1.72,0.32
2,00129213,80.89,212.27,5.13,0.60,2.79,8.18,0.00,0.00,0.00,4.83,7.70,1.50,38.39,60.46,1.95,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,00133173,486.84,0.00,472.06,388.09,224.17,200.57,181.96,115.26,0.00,199.02,163.49,160.73,178.80,140.58,197.99,0.00,165.40,66.73,53.37,47.26,224.66,189.91,0.00,180.94
4,00133210,0.00,0.00,1.00,2.00,1.00,3.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [49]:
first_month = 0
last_month = 23
serie_size = 12
data_series = []

for index, row in summed_data.iterrows():
    for month1 in range((last_month - (first_month + serie_size)) + 1):
        # serie = [row['barcode'], row['kodejenis']]
        serie = [row['barcode']]
        for month2 in range(serie_size + 1):
            serie.append(row[month1 + first_month + month2])
        data_series.append(serie)

columns = ['barcode']
[columns.append(i) for i in range(serie_size)]
columns.append('label')
data_series = pd.DataFrame(data_series, columns=columns)
data_series.head()

,barcode,0,1,2,3,4,5,6,7,8,9,10,11,label
0,00100809,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23
1,00100809,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75
2,00100809,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23
3,00100809,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37
4,00100809,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12


In [50]:
data_series = data_series.drop(['barcode'], axis=1)
data_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23
1,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75
2,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23
3,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37
4,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12


In [51]:
# outname = 'data_series_2020_2021.csv'
# outname = 'data_series_2020_2021_limit200.csv'
# outname = 'data_series_2020_2021_limit300.csv'
# outname = 'data_series_2020_2021_limit400.csv'
outname = 'data_series_2020_2021_limit500.csv'

outdir = './data-train-unlimit'

if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

data_series.to_csv(fullname)

# YANG DIBAWAH ENGGA JADI DIPAKAI

# 2017-2021 Without Kategori

In [ ]:
df2017 = pd.read_csv('./data-2017-renew.csv', parse_dates=['tgl'])
df2018 = pd.read_csv('./data-2018-renew.csv', parse_dates=['tgl'])
df2019 = pd.read_csv('./data-2019-renew.csv', parse_dates=['tgl'])
df2020 = pd.read_csv('./data-2020-renew.csv', parse_dates=['tgl'])
df2021 = pd.read_csv('./data-2021-renew.csv', parse_dates=['tgl'])

In [ ]:
# get unique kategori value

group_kategori2017 = df2017.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2018 = df2018.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2019 = df2019.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2020 = df2020.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2021 = df2021.groupby('kodekategori')['kategori'].unique().reset_index()

pdList = [group_kategori2017, group_kategori2018, group_kategori2019, group_kategori2020, group_kategori2021]

new_kategori = pd.concat(pdList).reset_index(drop=True)
new_kategori['kodekategori'] = new_kategori['kodekategori'].astype(float)
new_kategori['kategori'] = new_kategori['kategori'].astype(str)

clean_kategori = new_kategori.drop_duplicates(subset=['kodekategori']).reset_index(drop=True)

In [ ]:
display(clean_kategori.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115
kodekategori,101.00,102.00,103.00,104.00,105.00,106.00,107.00,108.00,109.00,110.00,111.00,112.00,113.00,114.00,115.00,201.00,202.00,203.00,204.00,205.00,206.00,301.00,302.00,10101.00,10102.00,10103.00,10104.00,10105.00,10106.00,10107.00,10108.00,10109.00,10110.00,10111.00,10112.00,10113.00,10114.00,10115.00,10116.00,10117.00,10118.00,10119.00,10120.00,10121.00,10201.00,10202.00,10203.00,10204.00,10205.00,10206.00,10207.00,10208.00,10209.00,10210.00,10211.00,10212.00,10213.00,20101.00,20201.00,20402.00,20501.00,20502.00,20503.00,20504.00,20601.00,30101.00,30201.00,30202.00,30203.00,30204.00,30301.00,30302.00,30303.00,30304.00,30305.00,30306.00,30307.00,30308.00,30309.00,30310.00,30311.00,30312.00,30401.00,30402.00,30403.00,30404.00,30405.00,30406.00,30407.00,40101.00,40103.00,40201.00,40202.00,40203.00,40301.00,40302.00,40303.00,40304.00,40305.00,40306.00,40307.00,40308.00,40309.00,40312.00,40401.00,40402.00,40403.00,40404.00,40405.00,20301.00,40310.00,40311.00,40501.00,40601.00,20302.00,40502.00
kategori,['Baby Needs'],['Mens Care (2)'],['Healt & Beauty Care (3)'],['Kids Care (4)'],['Cleaning (5)'],['Homes (6)'],['Electronic & Tool kits (7)'],['Automotive (8)'],['Stationery (9)'],['Toys (10)'],['Sports (11)'],['Event Equipment (12)'],['Convection (13)'],['Shoes & Sandals (14)'],['Bag & Luggage (15)'],['Food (1)'],['Beverage (2)'],['Medicine (3)'],['Pets food (4)'],['Cigarette (5)'],['Fresh Food (6)'],['Others (1)'],['alisha collection'],['Makanan Bayi (Baby Food)'],['Bahan Kue (Baking Supplies)'],['Biskuit (Biscuit)'],['Roti dan Kue (Bread & Cake)'],['Sarapan Pagi (Breakfast)'],['Penganan Permen (Candy Confectionary)'],['Penganan Coklat (Chocolate Confectionary)'],['Makanan Mentah (Raw Food)'],['Ice'],['Makanan Masak (Cooked Food)'],['Bumbu Dapur (Seasoning)'],['Mie (Noodle)'],['Pasta'],['Makanan Ringan (Snack)'],['Beras (Rice)'],['Biji-bijian (Seeds)'],['Gula (Sugar)'],['Tepung (Flour)'],['Minyak Goreng (Cooking Oil)'],['Makanan Kaleng (Canned Food)'],['Makanan Hewan (Pet Food)'],['Susu (Milk)'],['Kopi (Coffe)'],['Teh (Tea)'],['Air (Water)'],['Minuman Buah (Juice)'],['Minuman Kesehatan (Healty Drink)'],['Minuman Berenergi (Energy Drink)'],['Minuman Isotonik (Isotonik Drink)'],['Minuman Penyegar (Refreshment)'],['Minuman Jeli (Jelly Drink)'],['Minuman Bersoda (Soft Drink)'],['Minuman Beralkohol (Alcoholic Drink)'],['Sirup (Syrup)'],['Buah (Fruit)'],['Sayur (Vegetable)'],['Air Tawar'],['Susu & Olahan (Dairy)'],['Harian (Daily)'],['Beku (Frozen)'],['Telur (Egg)'],['Produk Siap Makan (Ready To Eat)'],['Rokok (Cigarete)'],['Obat Dalam (Peroral)'],['Obat Luar (Topikal)'],['Vitamin & Suplemen (Vitamis & Supplements)'],['Alat Kesehatan (Health Appliance)'],['Perawatan Rambut (Hair Care)'],['Perawatan Wajah (Face Care)'],['Kosmetik (Cosmetic)'],['Tisu & Kapas (Tissues & Cotton)'],['Perawatan Gigi & Mulut (Oral Care)'],['Perawatan Tubuh (Body Care)'],['Perawatan Bayi (Baby Care)'],['Perawatan Anak (Kids Care)'],['Popok Bayi (Baby Diapers)'],['Pembalut Wanita (Sanitary Napkins)'],['Popok Dewasa (Adult Diapers)'],['Perawatan Pria (Mens Groming)'],['Perawatan Pakaian (Clothes Care)'],['Perawatan Dapur (Kitchen Care)'],['Perawatan Rumah (House Care)'],['Pengharum Rumah (Home Fragrances)'],['Pembasmi Serangga & Pengendali Hama (Insecti...,['Perawatan Tas & Sepatu Kulit (Leather Care)'],['Perawatan Otomotif (Automotive Care)'],['Mainan Lunak (Soft Toys)'],['Mainan Keras (Hard Toys)'],['Peralatan (Tools)'],['Perlengkapan (Supplies)'],['Fancies'],['Rumah Tangga (House Hold)'],['Dapur (Kitchen)'],['Kamar Mandi (Bath Room)'],['Kamar Tidur (Bed Room)'],['Peralatan & Perlengkapan Bayi (Equipment & B

In [ ]:
# initiate the kategori list that will removed from dataset
kategori_remove_name = ["Fresh Food (6)", "Roti dan Kue (Bread & Cake)", "Sarapan Pagi (Breakfast)", "Makanan Mentah (Raw Food)", "Ice", "Makanan Masak (Cooked Food)", "Buah (Fruit)", "Sayur (Vegetable)", "Harian (Daily)", "Beku (Frozen)", "Telur (Egg)", "Produk Siap Makan (Ready To Eat)", "Daging Merah (Red Meat)", "Daging Putih (White Meat)"]
kategori_remove_list = ["206.00", "10104.00", "10105.00", "10108.00", "10109.00", "10110.00", "20101.00", "20201.00", "20502.00", "20503.00", "20504.00", "20601.00", "20301.00", "20302.00"]

In [ ]:
kategori_name_df = pd.DataFrame(kategori_remove_name)
# kategori_name_df.to_csv('kategori_delist.csv', index=False)

In [ ]:
# drop unnecessary column
df2017 = df2017.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2018 = df2018.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2019 = df2019.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2020 = df2020.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2021 = df2021.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)

In [ ]:
# concat 5 dataframe to single df
df = pd.concat([df2017, df2018, df2019, df2020, df2021]).reset_index(drop=True)

In [ ]:
# remove strange value in each rows from barcode column
df['barcode'] = df['barcode'].str.split(' ').str[0]

In [ ]:
df.head(10)

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori
0,2017-10-08 10:21:56.667,SHINZUI SOAP REGULAR75GR*144,PCS,2.00,8992946511790,CU00000001,10301,Bath Care (1),103.00,Healt & Beauty Care (3)
1,2017-06-10 15:14:28.343,KOL PUTIH/KG,KG,0.19,00172677,CU00000001,20607,Vegetable (7),206.00,Fresh Food (6)
2,2017-09-18 20:38:05.290,PASEO SMART TOILET WHT 6143*64,PCS,1.00,8993053211672,CU00000001,10507,Tissue & Cotton (7),105.00,Cleaning (5)
3,2017-02-03 19:58:25.060,POND'S WB NIGHT CREAM 20GR/48,PCS,1.00,8999999056872,CU00000001,10302,Cosmetic & Accessories (2),103.00,Healt & Beauty Care (3)
4,2017-05-27 15:44:34.500,INDOMIE RASA SOTO SPESIAL 75GR,PCS,2.00,089686910384,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1)
5,2017-12-03 17:01:00.420,TB L/XL UK 30,PCS,1.00,56103,CU00000001,10601,Home Applience (1,106.00,Homes (6)
6,2017-10-31 15:57:49.297,INDOMIE KALDU AYAM,PCS,2.00,089686010107,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1)
7,2017-03-10 14:03:49.983,ROYCO FDS BEEF 100GR,PCS,1.00,8999999192198,CU00000001,20117,Seasoning (17),201.00,Food (1)
8,2017-07-07 18:50:42.257,PEPSODENT SNSTV EXPERT WHT 100GR,PCS,1.00,8999999032890,CU00000001,10307,Oral Care (7),103.00,Healt & Beauty Care (3)
9,2017-03-02 20:26:00.757,MOLTO EDP BLACK PURPLE 22ML/216 (18),BDD,2.00,8999999057640,CU00000001,10506,Softener (6),105.00,Cleaning (5)


In [ ]:
# drop unnesecary value from dataset 
df = df[df['kodecustomer'].isin(['CU00000001'])].reset_index(drop=True)
df = df[~df['barcode'].isin(['56101','56102','56103','56104'])].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('JASA PENDINGIN')].reset_index(drop=True)
df = df[~df['kategori'].str.contains('Toys', na=False)].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('ZANDILAC')].reset_index(drop=True)

In [ ]:
# create new column named "month_year" to get number of month and year
df['month_year'] = df['tgl'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [ ]:
# transform the value to number of month from all rows
# 2017-2021
df['month_year'] = df['month_year'].map({'01-17':0,'02-17':1,'03-17':2,'04-17':3,'05-17':4,'06-17':5,'07-17':6,'08-17':7,'09-17':8,'10-17':9,'11-17':10,'12-17':11,
                                          '01-18':12,'02-18':13,'03-18':14,'04-18':15,'05-18':16,'06-18':17,'07-18':18,'08-18':19,'09-18':20,'10-18':21,'11-18':22,'12-18':23,
                                          '01-19':24,'02-19':25,'03-19':26,'04-19':27,'05-19':28,'06-19':29,'07-19':30,'08-19':31,'09-19':32,'10-19':33,'11-19':34,'12-19':35,
                                          '01-20':36,'02-20':37,'03-20':38,'04-20':39,'05-20':40,'06-20':41,'07-20':42,'08-20':43,'09-20':44,'10-20':45,'11-20':46,'12-20':47,
                                          '01-21':48,'02-21':49,'03-21':50,'04-21':51,'05-21':52,'06-21':53,'07-21':54,'08-21':55,'09-21':56,'10-21':57,'11-21':58,'12-21':59})

In [ ]:
# drop unnecessary kategori #tidak diremove
df = df[~df['kategori'].isin(kategori_remove_name)].reset_index(drop=True)

In [ ]:
display(df.info())
display(df.head())
# RangeIndex: 11533836 entries, 0 to 11533835 sebelum nambah daftar kategori

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10705848 entries, 0 to 10705847
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   tgl           datetime64[ns]
 1   namabarang    object        
 2   satuan        object        
 3   qty           float64       
 4   barcode       object        
 5   kodecustomer  object        
 6   kodejenis     object        
 7   jenis         object        
 8   kodekategori  float64       
 9   kategori      object        
 10  month_year    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 898.5+ MB


None

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori,month_year
0,2017-10-08 10:21:56.667,SHINZUI SOAP REGULAR75GR*144,PCS,2.00,8992946511790,CU00000001,10301,Bath Care (1),103.00,Healt & Beauty Care (3),9
1,2017-09-18 20:38:05.290,PASEO SMART TOILET WHT 6143*64,PCS,1.00,8993053211672,CU00000001,10507,Tissue & Cotton (7),105.00,Cleaning (5),8
2,2017-02-03 19:58:25.060,POND'S WB NIGHT CREAM 20GR/48,PCS,1.00,8999999056872,CU00000001,10302,Cosmetic & Accessories (2),103.00,Healt & Beauty Care (3),1
3,2017-05-27 15:44:34.500,INDOMIE RASA SOTO SPESIAL 75GR,PCS,2.00,089686910384,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1),4
4,2017-10-31 15:57:49.297,INDOMIE KALDU AYAM,PCS,2.00,089686010107,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1),9


In [ ]:
train_monthly = df[['tgl', 'month_year', 'kodejenis', 'barcode', 'qty']].copy()
train_monthly = train_monthly.sort_values('tgl').groupby(['month_year', 'kodejenis', 'barcode'], as_index=False)
train_monthly = train_monthly.agg({'qty':['sum']})
train_monthly.columns = ['month_year', 'kodejenis', 'barcode', 'qty_cnt']
train_monthly = train_monthly.query('qty_cnt > 0')

train_monthly['qty_cnt_month'] = train_monthly.sort_values('month_year').groupby(['barcode', 'kodejenis'])['qty_cnt'].shift(-1)

display(train_monthly.head())

,month_year,kodejenis,barcode,qty_cnt,qty_cnt_month
0,0,10101,4902430600859,2.00,NaN
1,0,10101,4902430601191,4.00,2.00
2,0,10101,4902430601238,4.00,1.00
3,0,10101,4902430601245,7.00,NaN
4,0,10101,4902430601320,1.00,1.00


In [ ]:
monthly_series = train_monthly.pivot_table(index=['barcode', 'kodejenis'], columns='month_year',values='qty_cnt', fill_value=0).reset_index()
monthly_series.head()

month_year,barcode,kodejenis,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,00100809,01011601,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00166454,01011109,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.24,1.14,0.96,0.66,0.81,0.71,1.29,0.55,0.67,0.70,0.77,1.51,1.39,0.30,0.40,0.27,0.35,0.27,0.14,0.17,0.13,0.07,0.60,0.11,0.27,0.00,0.22,0.17,0.17,0.33,0.51,0.29,0.05,0.36
2,00166461,01011109,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.83,12.62,11.39,9.76,7.74,6.45,6.24,8.99,9.01,4.83,8.03,8.10,7.14,9.01,29.51,9.92,5.89,6.63,2.98,4.06,4.44,4.54,1.56,2.67,3.03,4.35,3.56,2.23,3.61,2.73,5.10,6.68,5.16,6.00,4.11,2.94
3,00166478,01011109,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.42,1.68,1.29,1.16,2.04,1.66,0.82,0.49,0.77,1.54,0.33,0.97,1.06,0.64,2.01,1.11,0.86,1.34,0.67,0.15,0.42,0.41,0.05,0.58,0.70,0.17,0.77,0.21,0.34,0.33,0.00,0.58,1.12,0.23,0.81,0.16
4,00166485,01011109,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.82,1.73,2.58,2.75,1.08,0.64,1.28,1.01,1.57,1.43,0.65,1.43,1.70,3.10,4.53,2.33,1.93,0.17,0.43,0.17,0.54,0.37,0.11,0.08,0.30,0.78,0.63,0.20,0.57,0.35,0.49,0.21,0.95,0.42,0.28,0.91


In [ ]:
monthly_series_drop = monthly_series.drop(['kodejenis'], axis=1)

In [ ]:
summed_data = monthly_series_drop.groupby('barcode').agg('sum').reset_index(False)
summed_data.head()

month_year,barcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59
0,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00166454,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.24,1.14,0.96,0.66,0.81,0.71,1.29,0.55,0.67,0.70,0.77,1.51,1.39,0.30,0.40,0.27,0.35,0.27,0.14,0.17,0.13,0.07,0.60,0.11,0.27,0.00,0.22,0.17,0.17,0.33,0.51,0.29,0.05,0.36
2,00166461,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.83,12.62,11.39,9.76,7.74,6.45,6.24,8.99,9.01,4.83,8.03,8.10,7.14,9.01,29.51,9.92,5.89,6.63,2.98,4.06,4.44,4.54,1.56,2.67,3.03,4.35,3.56,2.23,3.61,2.73,5.10,6.68,5.16,6.00,4.11,2.94
3,00166478,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.42,1.68,1.29,1.16,2.04,1.66,0.82,0.49,0.77,1.54,0.33,0.97,1.06,0.64,2.01,1.11,0.86,1.34,0.67,0.15,0.42,0.41,0.05,0.58,0.70,0.17,0.77,0.21,0.34,0.33,0.00,0.58,1.12,0.23,0.81,0.16
4,00166485,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.82,1.73,2.58,2.75,1.08,0.64,1.28,1.01,1.57,1.43,0.65,1.43,1.70,3.10,4.53,2.33,1.93,0.17,0.43,0.17,0.54,0.37,0.11,0.08,0.30,0.78,0.63,0.20,0.57,0.35,0.49,0.21,0.95,0.42,0.28,0.91


In [ ]:
first_month = 0
last_month = 59
serie_size = 12
data_series = []

for index, row in summed_data.iterrows():
    for month1 in range((last_month - (first_month + serie_size)) + 1):
        # serie = [row['barcode'], row['kodejenis']]
        serie = [row['barcode']]
        for month2 in range(serie_size + 1):
            serie.append(row[month1 + first_month + month2])
        data_series.append(serie)

columns = ['barcode']
[columns.append(i) for i in range(serie_size)]
columns.append('label')
data_series = pd.DataFrame(data_series, columns=columns)
data_series.head()

,barcode,0,1,2,3,4,5,6,7,8,9,10,11,label
0,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
data_series = data_series.drop(['barcode'], axis=1)
data_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
outname = 'data_series_2017_2021_without_kategori.csv'
outdir = './data-train-unlimit'

if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

data_series.to_csv(fullname)

# 2018-2021 Without Kategori

In [ ]:
df2018 = pd.read_csv('./data-2018-renew.csv', parse_dates=['tgl'])
df2019 = pd.read_csv('./data-2019-renew.csv', parse_dates=['tgl'])
df2020 = pd.read_csv('./data-2020-renew.csv', parse_dates=['tgl'])
df2021 = pd.read_csv('./data-2021-renew.csv', parse_dates=['tgl'])

In [ ]:
# get unique kategori value

group_kategori2018 = df2018.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2019 = df2019.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2020 = df2020.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2021 = df2021.groupby('kodekategori')['kategori'].unique().reset_index()

pdList = [group_kategori2018, group_kategori2019, group_kategori2020, group_kategori2021]

new_kategori = pd.concat(pdList).reset_index(drop=True)
new_kategori['kodekategori'] = new_kategori['kodekategori'].astype(float)
new_kategori['kategori'] = new_kategori['kategori'].astype(str)

clean_kategori = new_kategori.drop_duplicates(subset=['kodekategori']).reset_index(drop=True)

In [ ]:
display(clean_kategori.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115
kodekategori,101.00,102.00,103.00,104.00,105.00,106.00,107.00,108.00,109.00,110.00,111.00,112.00,113.00,114.00,115.00,201.00,202.00,203.00,204.00,205.00,206.00,301.00,302.00,10101.00,10102.00,10103.00,10104.00,10105.00,10106.00,10107.00,10108.00,10109.00,10110.00,10111.00,10112.00,10113.00,10114.00,10115.00,10116.00,10117.00,10118.00,10119.00,10120.00,10121.00,10201.00,10202.00,10203.00,10204.00,10205.00,10206.00,10207.00,10208.00,10209.00,10210.00,10211.00,10212.00,10213.00,20101.00,20201.00,20402.00,20501.00,20502.00,20503.00,20504.00,20601.00,30101.00,30201.00,30202.00,30203.00,30204.00,30301.00,30302.00,30303.00,30304.00,30305.00,30306.00,30307.00,30308.00,30309.00,30310.00,30311.00,30312.00,30401.00,30402.00,30403.00,30404.00,30405.00,30406.00,30407.00,40101.00,40103.00,40201.00,40202.00,40203.00,40301.00,40302.00,40303.00,40304.00,40305.00,40306.00,40307.00,40308.00,40309.00,40312.00,40401.00,40402.00,40403.00,40404.00,40405.00,20301.00,40310.00,40311.00,40501.00,40601.00,20302.00,40502.00
kategori,['Baby Needs'],['Mens Care (2)'],['Healt & Beauty Care (3)'],['Kids Care (4)'],['Cleaning (5)'],['Homes (6)'],['Electronic & Tool kits (7)'],['Automotive (8)'],['Stationery (9)'],['Toys (10)'],['Sports (11)'],['Event Equipment (12)'],['Convection (13)'],['Shoes & Sandals (14)'],['Bag & Luggage (15)'],['Food (1)'],['Beverage (2)'],['Medicine (3)'],['Pets food (4)'],['Cigarette (5)'],['Fresh Food (6)'],['Others (1)'],['alisha collection'],['Makanan Bayi (Baby Food)'],['Bahan Kue (Baking Supplies)'],['Biskuit (Biscuit)'],['Roti dan Kue (Bread & Cake)'],['Sarapan Pagi (Breakfast)'],['Penganan Permen (Candy Confectionary)'],['Penganan Coklat (Chocolate Confectionary)'],['Makanan Mentah (Raw Food)'],['Ice'],['Makanan Masak (Cooked Food)'],['Bumbu Dapur (Seasoning)'],['Mie (Noodle)'],['Pasta'],['Makanan Ringan (Snack)'],['Beras (Rice)'],['Biji-bijian (Seeds)'],['Gula (Sugar)'],['Tepung (Flour)'],['Minyak Goreng (Cooking Oil)'],['Makanan Kaleng (Canned Food)'],['Makanan Hewan (Pet Food)'],['Susu (Milk)'],['Kopi (Coffe)'],['Teh (Tea)'],['Air (Water)'],['Minuman Buah (Juice)'],['Minuman Kesehatan (Healty Drink)'],['Minuman Berenergi (Energy Drink)'],['Minuman Isotonik (Isotonik Drink)'],['Minuman Penyegar (Refreshment)'],['Minuman Jeli (Jelly Drink)'],['Minuman Bersoda (Soft Drink)'],['Minuman Beralkohol (Alcoholic Drink)'],['Sirup (Syrup)'],['Buah (Fruit)'],['Sayur (Vegetable)'],['Air Tawar'],['Susu & Olahan (Dairy)'],['Harian (Daily)'],['Beku (Frozen)'],['Telur (Egg)'],['Produk Siap Makan (Ready To Eat)'],['Rokok (Cigarete)'],['Obat Dalam (Peroral)'],['Obat Luar (Topikal)'],['Vitamin & Suplemen (Vitamis & Supplements)'],['Alat Kesehatan (Health Appliance)'],['Perawatan Rambut (Hair Care)'],['Perawatan Wajah (Face Care)'],['Kosmetik (Cosmetic)'],['Tisu & Kapas (Tissues & Cotton)'],['Perawatan Gigi & Mulut (Oral Care)'],['Perawatan Tubuh (Body Care)'],['Perawatan Bayi (Baby Care)'],['Perawatan Anak (Kids Care)'],['Popok Bayi (Baby Diapers)'],['Pembalut Wanita (Sanitary Napkins)'],['Popok Dewasa (Adult Diapers)'],['Perawatan Pria (Mens Groming)'],['Perawatan Pakaian (Clothes Care)'],['Perawatan Dapur (Kitchen Care)'],['Perawatan Rumah (House Care)'],['Pengharum Rumah (Home Fragrances)'],['Pembasmi Serangga & Pengendali Hama (Insecti...,['Perawatan Tas & Sepatu Kulit (Leather Care)'],['Perawatan Otomotif (Automotive Care)'],['Mainan Lunak (Soft Toys)'],['Mainan Keras (Hard Toys)'],['Peralatan (Tools)'],['Perlengkapan (Supplies)'],['Fancies'],['Rumah Tangga (House Hold)'],['Dapur (Kitchen)'],['Kamar Mandi (Bath Room)'],['Kamar Tidur (Bed Room)'],['Peralatan & Perlengkapan Bayi (Equipment & B

In [ ]:
# initiate the kategori list that will removed from dataset
kategori_remove_name = ["Fresh Food (6)", "Roti dan Kue (Bread & Cake)", "Sarapan Pagi (Breakfast)", "Makanan Mentah (Raw Food)", "Ice", "Makanan Masak (Cooked Food)", "Buah (Fruit)", "Sayur (Vegetable)", "Harian (Daily)", "Beku (Frozen)", "Telur (Egg)", "Produk Siap Makan (Ready To Eat)", "Daging Merah (Red Meat)", "Daging Putih (White Meat)"]
kategori_remove_list = ["206.00", "10104.00", "10105.00", "10108.00", "10109.00", "10110.00", "20101.00", "20201.00", "20502.00", "20503.00", "20504.00", "20601.00", "20301.00", "20302.00"]

In [ ]:
kategori_name_df = pd.DataFrame(kategori_remove_name)
# kategori_name_df.to_csv('kategori_delist.csv', index=False)

In [ ]:
# drop unnecessary column
df2018 = df2018.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2019 = df2019.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2020 = df2020.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2021 = df2021.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)

In [ ]:
# concat 5 dataframe to single df
df = pd.concat([df2018, df2019, df2020, df2021]).reset_index(drop=True)

In [ ]:
# remove strange value in each rows from barcode column
df['barcode'] = df['barcode'].str.split(' ').str[0]

In [ ]:
df.head(10)

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori
0,2018-04-18 19:18:51.523,TB S/M UK 25,PCS,1.00,56102,CU00000001,10601,Home Applience (1,106.00,Homes (6)
1,2018-11-19 18:19:38.213,INDOMIE GORENG KRIUK PEDAS,PCS,1.00,089686059779,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1)
2,2018-10-12 17:31:53.357,SARIMI RASA AYAM (RRABG),PCS,16.00,089686017700,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1)
3,2018-07-15 18:36:40.493,VAPE MAT ULTRA 60'S/45'S,PCS,1.00,8992857050135,CU00000001,10505,Pes Control (5),105.00,Cleaning (5)
4,2018-12-08 21:33:23.330,PENGHAPUS STEADLER 526 B40*40,PCS,1.00,4007817524206,CU00000001,10903,Stationery (3),109.00,Stationery (9)
5,2018-03-25 20:22:25.703,CHARM L&W BREATHABLE NP 40P,PCS,1.00,8993189272066,CU00000001,10306,Napkin (6),103.00,Healt & Beauty Care (3)
6,2018-07-04 12:36:03.487,ROK SUN SHINE 1208,PCS,1.00,85738,CU00000001,11303,Kids Wear (3),113.00,Convection (13)
7,2018-06-21 18:57:34.150,SUNLIGHT LIME REF 240ML/24,PCS,1.00,8999999059781,CU00000001,10503,Home & Leater Care (3),105.00,Cleaning (5)
8,2018-10-31 20:53:14.920,BIORE BF LOVELY SAKURA POUCH 250ML*24,PCS,1.00,8992727006170,CU00000001,10301,Bath Care (1),103.00,Healt & Beauty Care (3)
9,2018-12-11 13:47:03.247,"PARUTAN MENARA TG 18"" BIASA",PCS,1.00,13238,CU00000001,10602,Kitchen (2),106.00,Homes (6)


In [ ]:
# drop unnesecary value from dataset 
df = df[df['kodecustomer'].isin(['CU00000001'])].reset_index(drop=True)
df = df[~df['barcode'].isin(['56101','56102','56103','56104'])].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('JASA PENDINGIN')].reset_index(drop=True)
df = df[~df['kategori'].str.contains('Toys', na=False)].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('ZANDILAC')].reset_index(drop=True)

In [ ]:
# create new column named "month_year" to get number of month and year
df['month_year'] = df['tgl'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [ ]:
# transform the value to number of month from all rows
2018-2021
df['month_year'] = df['month_year'].map({'01-18':0,'02-18':1,'03-18':2,'04-18':3,'05-18':4,'06-18':5,'07-18':6,'08-18':7,'09-18':8,'10-18':9,'11-18':10,'12-18':11,
                                          '01-19':12,'02-19':13,'03-19':14,'04-19':15,'05-19':16,'06-19':17,'07-19':18,'08-19':19,'09-19':20,'10-19':21,'11-19':22,'12-19':23,
                                          '01-20':24,'02-20':25,'03-20':26,'04-20':27,'05-20':28,'06-20':29,'07-20':30,'08-20':31,'09-20':32,'10-20':33,'11-20':34,'12-20':35,
                                          '01-21':36,'02-21':37,'03-21':38,'04-21':39,'05-21':40,'06-21':41,'07-21':42,'08-21':43,'09-21':44,'10-21':45,'11-21':46,'12-21':47})

In [ ]:
# drop unnecessary kategori #tidak diremove
df = df[~df['kategori'].isin(kategori_remove_name)].reset_index(drop=True)

In [ ]:
display(df.info())
display(df.head())
# RangeIndex: 11533836 entries, 0 to 11533835 sebelum nambah daftar kategori

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8291782 entries, 0 to 8291781
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   tgl           datetime64[ns]
 1   namabarang    object        
 2   satuan        object        
 3   qty           float64       
 4   barcode       object        
 5   kodecustomer  object        
 6   kodejenis     object        
 7   jenis         object        
 8   kodekategori  float64       
 9   kategori      object        
 10  month_year    int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 695.9+ MB


None

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori,month_year
0,2018-11-19 18:19:38.213,INDOMIE GORENG KRIUK PEDAS,PCS,1.00,089686059779,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1),10
1,2018-10-12 17:31:53.357,SARIMI RASA AYAM (RRABG),PCS,16.00,089686017700,CU00000001,20113,Noodle & Pasta (13),201.00,Food (1),9
2,2018-07-15 18:36:40.493,VAPE MAT ULTRA 60'S/45'S,PCS,1.00,8992857050135,CU00000001,10505,Pes Control (5),105.00,Cleaning (5),6
3,2018-12-08 21:33:23.330,PENGHAPUS STEADLER 526 B40*40,PCS,1.00,4007817524206,CU00000001,10903,Stationery (3),109.00,Stationery (9),11
4,2018-03-25 20:22:25.703,CHARM L&W BREATHABLE NP 40P,PCS,1.00,8993189272066,CU00000001,10306,Napkin (6),103.00,Healt & Beauty Care (3),2


In [ ]:
train_monthly = df[['tgl', 'month_year', 'kodejenis', 'barcode', 'qty']].copy()
train_monthly = train_monthly.sort_values('tgl').groupby(['month_year', 'kodejenis', 'barcode'], as_index=False)
train_monthly = train_monthly.agg({'qty':['sum']})
train_monthly.columns = ['month_year', 'kodejenis', 'barcode', 'qty_cnt']
train_monthly = train_monthly.query('qty_cnt > 0')

train_monthly['qty_cnt_month'] = train_monthly.sort_values('month_year').groupby(['barcode', 'kodejenis'])['qty_cnt'].shift(-1)

display(train_monthly.head())

,month_year,kodejenis,barcode,qty_cnt,qty_cnt_month
0,0,10101,089686960044,2.00,3.00
1,0,10101,089686960075,1.00,2.00
2,0,10101,089686960396,5.00,8.00
3,0,10101,4902430605144,4.00,7.00
4,0,10101,4902430610346,2.00,3.00


In [ ]:
monthly_series = train_monthly.pivot_table(index=['barcode', 'kodejenis'], columns='month_year',values='qty_cnt', fill_value=0).reset_index()
monthly_series.head()

month_year,barcode,kodejenis,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,00100809,01011601,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00166454,01011109,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.24,1.14,0.96,0.66,0.81,0.71,1.29,0.55,0.67,0.70,0.77,1.51,1.39,0.30,0.40,0.27,0.35,0.27,0.14,0.17,0.13,0.07,0.60,0.11,0.27,0.00,0.22,0.17,0.17,0.33,0.51,0.29,0.05,0.36
2,00166461,01011109,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.83,12.62,11.39,9.76,7.74,6.45,6.24,8.99,9.01,4.83,8.03,8.10,7.14,9.01,29.51,9.92,5.89,6.63,2.98,4.06,4.44,4.54,1.56,2.67,3.03,4.35,3.56,2.23,3.61,2.73,5.10,6.68,5.16,6.00,4.11,2.94
3,00166478,01011109,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.42,1.68,1.29,1.16,2.04,1.66,0.82,0.49,0.77,1.54,0.33,0.97,1.06,0.64,2.01,1.11,0.86,1.34,0.67,0.15,0.42,0.41,0.05,0.58,0.70,0.17,0.77,0.21,0.34,0.33,0.00,0.58,1.12,0.23,0.81,0.16
4,00166485,01011109,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.82,1.73,2.58,2.75,1.08,0.64,1.28,1.01,1.57,1.43,0.65,1.43,1.70,3.10,4.53,2.33,1.93,0.17,0.43,0.17,0.54,0.37,0.11,0.08,0.30,0.78,0.63,0.20,0.57,0.35,0.49,0.21,0.95,0.42,0.28,0.91


In [ ]:
monthly_series_drop = monthly_series.drop(['kodejenis'], axis=1)

In [ ]:
summed_data = monthly_series_drop.groupby('barcode').agg('sum').reset_index(False)
summed_data.head()

month_year,barcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00166454,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,1.24,1.14,0.96,0.66,0.81,0.71,1.29,0.55,0.67,0.70,0.77,1.51,1.39,0.30,0.40,0.27,0.35,0.27,0.14,0.17,0.13,0.07,0.60,0.11,0.27,0.00,0.22,0.17,0.17,0.33,0.51,0.29,0.05,0.36
2,00166461,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13.83,12.62,11.39,9.76,7.74,6.45,6.24,8.99,9.01,4.83,8.03,8.10,7.14,9.01,29.51,9.92,5.89,6.63,2.98,4.06,4.44,4.54,1.56,2.67,3.03,4.35,3.56,2.23,3.61,2.73,5.10,6.68,5.16,6.00,4.11,2.94
3,00166478,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.42,1.68,1.29,1.16,2.04,1.66,0.82,0.49,0.77,1.54,0.33,0.97,1.06,0.64,2.01,1.11,0.86,1.34,0.67,0.15,0.42,0.41,0.05,0.58,0.70,0.17,0.77,0.21,0.34,0.33,0.00,0.58,1.12,0.23,0.81,0.16
4,00166485,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.82,1.73,2.58,2.75,1.08,0.64,1.28,1.01,1.57,1.43,0.65,1.43,1.70,3.10,4.53,2.33,1.93,0.17,0.43,0.17,0.54,0.37,0.11,0.08,0.30,0.78,0.63,0.20,0.57,0.35,0.49,0.21,0.95,0.42,0.28,0.91


In [ ]:
first_month = 0
last_month = 47
serie_size = 12
data_series = []

for index, row in summed_data.iterrows():
    for month1 in range((last_month - (first_month + serie_size)) + 1):
        # serie = [row['barcode'], row['kodejenis']]
        serie = [row['barcode']]
        for month2 in range(serie_size + 1):
            serie.append(row[month1 + first_month + month2])
        data_series.append(serie)

columns = ['barcode']
[columns.append(i) for i in range(serie_size)]
columns.append('label')
data_series = pd.DataFrame(data_series, columns=columns)
data_series.head()

,barcode,0,1,2,3,4,5,6,7,8,9,10,11,label
0,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69
1,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35
2,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38
3,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80
4,00100809,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80,7.03


In [ ]:
data_series = data_series.drop(['barcode'], axis=1)
data_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,18.69,15.35,20.38,13.80,7.03


In [ ]:
outname = 'data_series_2018_2021_without_kategori.csv'
outdir = './data-train-unlimit'

if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

data_series.to_csv(fullname)

# 2019-2021 Without Kategori

In [ ]:
df2019 = pd.read_csv('./data-2019-renew.csv', parse_dates=['tgl'])
df2020 = pd.read_csv('./data-2020-renew.csv', parse_dates=['tgl'])
df2021 = pd.read_csv('./data-2021-renew.csv', parse_dates=['tgl'])

In [ ]:
# get unique kategori value

group_kategori2019 = df2019.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2020 = df2020.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2021 = df2021.groupby('kodekategori')['kategori'].unique().reset_index()

pdList = [group_kategori2019, group_kategori2020, group_kategori2021]

new_kategori = pd.concat(pdList).reset_index(drop=True)
new_kategori['kodekategori'] = new_kategori['kodekategori'].astype(float)
new_kategori['kategori'] = new_kategori['kategori'].astype(str)

clean_kategori = new_kategori.drop_duplicates(subset=['kodekategori']).reset_index(drop=True)

In [ ]:
display(clean_kategori.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115
kodekategori,101.00,102.00,103.00,104.00,105.00,106.00,107.00,108.00,109.00,110.00,111.00,112.00,113.00,114.00,115.00,201.00,202.00,203.00,205.00,206.00,301.00,302.00,10101.00,10102.00,10103.00,10104.00,10105.00,10106.00,10107.00,10108.00,10109.00,10110.00,10111.00,10112.00,10113.00,10114.00,10115.00,10116.00,10117.00,10118.00,10119.00,10120.00,10121.00,10201.00,10202.00,10203.00,10204.00,10205.00,10206.00,10207.00,10208.00,10209.00,10210.00,10211.00,10212.00,10213.00,20101.00,20201.00,20402.00,20501.00,20502.00,20503.00,20504.00,20601.00,30101.00,30201.00,30202.00,30203.00,30204.00,30301.00,30302.00,30303.00,30304.00,30305.00,30306.00,30307.00,30308.00,30309.00,30310.00,30311.00,30312.00,30401.00,30402.00,30403.00,30404.00,30405.00,30406.00,30407.00,40101.00,40103.00,40201.00,40202.00,40203.00,40301.00,40302.00,40303.00,40304.00,40305.00,40306.00,40307.00,40308.00,40309.00,40312.00,40401.00,40402.00,40403.00,40404.00,40405.00,204.00,20301.00,40310.00,40311.00,40501.00,40601.00,20302.00,40502.00
kategori,['Baby Needs'],['Mens Care (2)'],['Healt & Beauty Care (3)'],['Kids Care (4)'],['Cleaning (5)'],['Homes (6)'],['Electronic & Tool kits (7)'],['Automotive (8)'],['Stationery (9)'],['Toys (10)'],['Sports (11)'],['Event Equipment (12)'],['Convection (13)'],['Shoes & Sandals (14)'],['Bag & Luggage (15)'],['Food (1)'],['Beverage (2)'],['Medicine (3)'],['Cigarette (5)'],['Fresh Food (6)'],['Others (1)'],['alisha collection'],['Makanan Bayi (Baby Food)'],['Bahan Kue (Baking Supplies)'],['Biskuit (Biscuit)'],['Roti dan Kue (Bread & Cake)'],['Sarapan Pagi (Breakfast)'],['Penganan Permen (Candy Confectionary)'],['Penganan Coklat (Chocolate Confectionary)'],['Makanan Mentah (Raw Food)'],['Ice'],['Makanan Masak (Cooked Food)'],['Bumbu Dapur (Seasoning)'],['Mie (Noodle)'],['Pasta'],['Makanan Ringan (Snack)'],['Beras (Rice)'],['Biji-bijian (Seeds)'],['Gula (Sugar)'],['Tepung (Flour)'],['Minyak Goreng (Cooking Oil)'],['Makanan Kaleng (Canned Food)'],['Makanan Hewan (Pet Food)'],['Susu (Milk)'],['Kopi (Coffe)'],['Teh (Tea)'],['Air (Water)'],['Minuman Buah (Juice)'],['Minuman Kesehatan (Healty Drink)'],['Minuman Berenergi (Energy Drink)'],['Minuman Isotonik (Isotonik Drink)'],['Minuman Penyegar (Refreshment)'],['Minuman Jeli (Jelly Drink)'],['Minuman Bersoda (Soft Drink)'],['Minuman Beralkohol (Alcoholic Drink)'],['Sirup (Syrup)'],['Buah (Fruit)'],['Sayur (Vegetable)'],['Air Tawar'],['Susu & Olahan (Dairy)'],['Harian (Daily)'],['Beku (Frozen)'],['Telur (Egg)'],['Produk Siap Makan (Ready To Eat)'],['Rokok (Cigarete)'],['Obat Dalam (Peroral)'],['Obat Luar (Topikal)'],['Vitamin & Suplemen (Vitamis & Supplements)'],['Alat Kesehatan (Health Appliance)'],['Perawatan Rambut (Hair Care)'],['Perawatan Wajah (Face Care)'],['Kosmetik (Cosmetic)'],['Tisu & Kapas (Tissues & Cotton)'],['Perawatan Gigi & Mulut (Oral Care)'],['Perawatan Tubuh (Body Care)'],['Perawatan Bayi (Baby Care)'],['Perawatan Anak (Kids Care)'],['Popok Bayi (Baby Diapers)'],['Pembalut Wanita (Sanitary Napkins)'],['Popok Dewasa (Adult Diapers)'],['Perawatan Pria (Mens Groming)'],['Perawatan Pakaian (Clothes Care)'],['Perawatan Dapur (Kitchen Care)'],['Perawatan Rumah (House Care)'],['Pengharum Rumah (Home Fragrances)'],['Pembasmi Serangga & Pengendali Hama (Insecti...,['Perawatan Tas & Sepatu Kulit (Leather Care)'],['Perawatan Otomotif (Automotive Care)'],['Mainan Lunak (Soft Toys)'],['Mainan Keras (Hard Toys)'],['Peralatan (Tools)'],['Perlengkapan (Supplies)'],['Fancies'],['Rumah Tangga (House Hold)'],['Dapur (Kitchen)'],['Kamar Mandi (Bath Room)'],['Kamar Tidur (Bed Room)'],['Peralatan & Perlengkapan Bayi (Equipment & B...,['Peralatan & 

In [ ]:
# initiate the kategori list that will removed from dataset
kategori_remove_name = ["Fresh Food (6)", "Roti dan Kue (Bread & Cake)", "Sarapan Pagi (Breakfast)", "Makanan Mentah (Raw Food)", "Ice", "Makanan Masak (Cooked Food)", "Buah (Fruit)", "Sayur (Vegetable)", "Harian (Daily)", "Beku (Frozen)", "Telur (Egg)", "Produk Siap Makan (Ready To Eat)", "Daging Merah (Red Meat)", "Daging Putih (White Meat)"]
kategori_remove_list = ["206.00", "10104.00", "10105.00", "10108.00", "10109.00", "10110.00", "20101.00", "20201.00", "20502.00", "20503.00", "20504.00", "20601.00", "20301.00", "20302.00"]

In [ ]:
kategori_name_df = pd.DataFrame(kategori_remove_name)
# kategori_name_df.to_csv('kategori_delist.csv', index=False)

In [ ]:
# drop unnecessary column
df2019 = df2019.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2020 = df2020.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2021 = df2021.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)

In [ ]:
# concat 5 dataframe to single df
# df = pd.concat([df2017, df2018, df2019, df2020, df2021]).reset_index(drop=True)
df = pd.concat([df2019, df2020, df2021]).reset_index(drop=True)

In [ ]:
# remove strange value in each rows from barcode column
df['barcode'] = df['barcode'].str.split(' ').str[0]

In [ ]:
df.head(10)

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori
0,2019-09-24 11:19:44.360,VIT 600ML*24,PCS,1.00,8992752011408,CU00000001,01020403,Air Mineral Kemasan Botol,10204,Air (Water)
1,2019-09-08 16:09:13.810,BEAR BRAND 195GR*30,KLG,3.00,8992696404441,CU00000001,01020109,Susu Cair (RTD),10201,Susu (Milk)
2,2019-01-22 18:53:17.497,INDOMIE GORENG IGA PENYET,PCS,1.00,089686043686,CU00000001,01011202,Mie Instant,10112,Mie (Noodle)
3,2019-01-28 18:08:18.700,GARUDA ATOM MANIS AJTM130GR*60,BKS,3.00,8992775204009,CU00000001,01011403,Kacang Modern,10114,Makanan Ringan (Snack)
4,2019-02-24 20:38:04.383,INDOMIE MIE GORENG KHAS ACEH 90GR*40,PCS,1.00,089686043204,CU00000001,01011202,Mie Instant,10112,Mie (Noodle)
5,2019-09-30 18:50:43.320,SAORI SAUS TIRAM 23GR*10,PCS,2.00,8992770094117,CU00000001,01011104,Bumbu Saus,10111,Bumbu Dapur (Seasoning)
6,2019-10-26 19:46:10.427,DETTOL DAS ENERGIZE 110GR* 3S,BDD,1.00,8993560024901,CU00000001,03030601,Sabun Mandi Batang Antiseptic,30306,Perawatan Tubuh (Body Care)
7,2019-09-12 18:15:20.927,SUNLIGHT LIME REF 220ML/24,PCS,1.00,8999999059781,CU00000001,03040201,Sabun Cair,30402,Perawatan Dapur (Kitchen Care)
8,2019-06-21 17:47:50.450,ENTRASOL GOLD VANILA 185GR,PCS,1.00,8992802618731,CU00000001,01020105,Susu Kalsium,10201,Susu (Milk)
9,2019-09-01 18:58:05.730,CHARM LONG&WIDE EXT.DRY NP 40P,PCS,1.00,8993189270765,CU00000001,03031001,Panty Liner,30310,Pembalut Wanita (Sanitary Napkins)


In [ ]:
# drop unnesecary value from dataset 
df = df[df['kodecustomer'].isin(['CU00000001'])].reset_index(drop=True)
df = df[~df['barcode'].isin(['56101','56102','56103','56104'])].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('JASA PENDINGIN')].reset_index(drop=True)
df = df[~df['kategori'].str.contains('Toys', na=False)].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('ZANDILAC')].reset_index(drop=True)

In [ ]:
# create new column named "month_year" to get number of month and year
df['month_year'] = df['tgl'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [ ]:
# transform the value to number of month from all rows
# 2019-2021
df['month_year'] = df['month_year'].map({'01-19':0,'02-19':1,'03-19':2,'04-19':3,'05-19':4,'06-19':5,'07-19':6,'08-19':7,'09-19':8,'10-19':9,'11-19':10,'12-19':11,
                                          '01-20':12,'02-20':13,'03-20':14,'04-20':15,'05-20':16,'06-20':17,'07-20':18,'08-20':19,'09-20':20,'10-20':21,'11-20':22,'12-20':23,
                                          '01-21':24,'02-21':25,'03-21':26,'04-21':27,'05-21':28,'06-21':29,'07-21':30,'08-21':31,'09-21':32,'10-21':33,'11-21':34,'12-21':35})

In [ ]:
# drop unnecessary kategori #tidak diremove
df = df[~df['kategori'].isin(kategori_remove_name)].reset_index(drop=True)

In [ ]:
display(df.info())
display(df.head())
# RangeIndex: 11533836 entries, 0 to 11533835 sebelum nambah daftar kategori

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5774326 entries, 0 to 5774325
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   tgl           datetime64[ns]
 1   namabarang    object        
 2   satuan        object        
 3   qty           float64       
 4   barcode       object        
 5   kodecustomer  object        
 6   kodejenis     object        
 7   jenis         object        
 8   kodekategori  int64         
 9   kategori      object        
 10  month_year    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 484.6+ MB


None

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori,month_year
0,2019-09-24 11:19:44.360,VIT 600ML*24,PCS,1.00,8992752011408,CU00000001,01020403,Air Mineral Kemasan Botol,10204,Air (Water),8
1,2019-09-08 16:09:13.810,BEAR BRAND 195GR*30,KLG,3.00,8992696404441,CU00000001,01020109,Susu Cair (RTD),10201,Susu (Milk),8
2,2019-01-22 18:53:17.497,INDOMIE GORENG IGA PENYET,PCS,1.00,089686043686,CU00000001,01011202,Mie Instant,10112,Mie (Noodle),0
3,2019-01-28 18:08:18.700,GARUDA ATOM MANIS AJTM130GR*60,BKS,3.00,8992775204009,CU00000001,01011403,Kacang Modern,10114,Makanan Ringan (Snack),0
4,2019-02-24 20:38:04.383,INDOMIE MIE GORENG KHAS ACEH 90GR*40,PCS,1.00,089686043204,CU00000001,01011202,Mie Instant,10112,Mie (Noodle),1


In [ ]:
train_monthly = df[['tgl', 'month_year', 'kodejenis', 'barcode', 'qty']].copy()
train_monthly = train_monthly.sort_values('tgl').groupby(['month_year', 'kodejenis', 'barcode'], as_index=False)
train_monthly = train_monthly.agg({'qty':['sum']})
train_monthly.columns = ['month_year', 'kodejenis', 'barcode', 'qty_cnt']
train_monthly = train_monthly.query('qty_cnt > 0')

train_monthly['qty_cnt_month'] = train_monthly.sort_values('month_year').groupby(['barcode', 'kodejenis'])['qty_cnt'].shift(-1)

display(train_monthly.head())

,month_year,kodejenis,barcode,qty_cnt,qty_cnt_month
0,0,01010101,089686640007,33.00,34.00
1,0,01010101,089686640014,29.00,34.00
2,0,01010101,089686646047,9.00,16.00
3,0,01010101,089686646061,3.00,10.00
4,0,01010101,089686646450,1.00,2.00


In [ ]:
monthly_series = train_monthly.pivot_table(index=['barcode', 'kodejenis'], columns='month_year',values='qty_cnt', fill_value=0).reset_index()
monthly_series.head()

month_year,barcode,kodejenis,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,00100809,01011601,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00166454,01011109,0.50,0.00,1.24,1.14,0.96,0.66,0.81,0.71,1.29,0.55,0.67,0.70,0.77,1.51,1.39,0.30,0.40,0.27,0.35,0.27,0.14,0.17,0.13,0.07,0.60,0.11,0.27,0.00,0.22,0.17,0.17,0.33,0.51,0.29,0.05,0.36
2,00166461,01011109,13.83,12.62,11.39,9.76,7.74,6.45,6.24,8.99,9.01,4.83,8.03,8.10,7.14,9.01,29.51,9.92,5.89,6.63,2.98,4.06,4.44,4.54,1.56,2.67,3.03,4.35,3.56,2.23,3.61,2.73,5.10,6.68,5.16,6.00,4.11,2.94
3,00166478,01011109,1.42,1.68,1.29,1.16,2.04,1.66,0.82,0.49,0.77,1.54,0.33,0.97,1.06,0.64,2.01,1.11,0.86,1.34,0.67,0.15,0.42,0.41,0.05,0.58,0.70,0.17,0.77,0.21,0.34,0.33,0.00,0.58,1.12,0.23,0.81,0.16
4,00166485,01011109,2.82,1.73,2.58,2.75,1.08,0.64,1.28,1.01,1.57,1.43,0.65,1.43,1.70,3.10,4.53,2.33,1.93,0.17,0.43,0.17,0.54,0.37,0.11,0.08,0.30,0.78,0.63,0.20,0.57,0.35,0.49,0.21,0.95,0.42,0.28,0.91


In [ ]:
monthly_series_drop = monthly_series.drop(['kodejenis'], axis=1)

In [ ]:
summed_data = monthly_series_drop.groupby('barcode').agg('sum').reset_index(False)
summed_data.head()

month_year,barcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
0,00100809,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00166454,0.50,0.00,1.24,1.14,0.96,0.66,0.81,0.71,1.29,0.55,0.67,0.70,0.77,1.51,1.39,0.30,0.40,0.27,0.35,0.27,0.14,0.17,0.13,0.07,0.60,0.11,0.27,0.00,0.22,0.17,0.17,0.33,0.51,0.29,0.05,0.36
2,00166461,13.83,12.62,11.39,9.76,7.74,6.45,6.24,8.99,9.01,4.83,8.03,8.10,7.14,9.01,29.51,9.92,5.89,6.63,2.98,4.06,4.44,4.54,1.56,2.67,3.03,4.35,3.56,2.23,3.61,2.73,5.10,6.68,5.16,6.00,4.11,2.94
3,00166478,1.42,1.68,1.29,1.16,2.04,1.66,0.82,0.49,0.77,1.54,0.33,0.97,1.06,0.64,2.01,1.11,0.86,1.34,0.67,0.15,0.42,0.41,0.05,0.58,0.70,0.17,0.77,0.21,0.34,0.33,0.00,0.58,1.12,0.23,0.81,0.16
4,00166485,2.82,1.73,2.58,2.75,1.08,0.64,1.28,1.01,1.57,1.43,0.65,1.43,1.70,3.10,4.53,2.33,1.93,0.17,0.43,0.17,0.54,0.37,0.11,0.08,0.30,0.78,0.63,0.20,0.57,0.35,0.49,0.21,0.95,0.42,0.28,0.91


In [ ]:
first_month = 0
last_month = 35
serie_size = 12
data_series = []

for index, row in summed_data.iterrows():
    for month1 in range((last_month - (first_month + serie_size)) + 1):
        # serie = [row['barcode'], row['kodejenis']]
        serie = [row['barcode']]
        for month2 in range(serie_size + 1):
            serie.append(row[month1 + first_month + month2])
        data_series.append(serie)

columns = ['barcode']
[columns.append(i) for i in range(serie_size)]
columns.append('label')
data_series = pd.DataFrame(data_series, columns=columns)
data_series.head()

,barcode,0,1,2,3,4,5,6,7,8,9,10,11,label
0,00100809,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75
1,00100809,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78
2,00100809,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13
3,00100809,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93
4,00100809,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57


In [ ]:
data_series = data_series.drop(['barcode'], axis=1)
data_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,18.69,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75
1,15.35,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78
2,20.38,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13
3,13.80,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93
4,7.03,8.72,11.13,11.36,16.29,7.11,10.22,11.02,12.75,10.78,27.13,32.93,18.57


In [ ]:
outname = 'data_series_2019_2021_without_kategori.csv'
outdir = './data-train-unlimit'

if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

data_series.to_csv(fullname)

# 2020-2021 Without Kategori

In [ ]:
df2020 = pd.read_csv('./data-2020-renew.csv', parse_dates=['tgl'])
df2021 = pd.read_csv('./data-2021-renew.csv', parse_dates=['tgl'])

In [ ]:
# get unique kategori value

group_kategori2020 = df2020.groupby('kodekategori')['kategori'].unique().reset_index()
group_kategori2021 = df2021.groupby('kodekategori')['kategori'].unique().reset_index()

pdList = [group_kategori2020, group_kategori2021]

new_kategori = pd.concat(pdList).reset_index(drop=True)
new_kategori['kodekategori'] = new_kategori['kodekategori'].astype(float)
new_kategori['kategori'] = new_kategori['kategori'].astype(str)

clean_kategori = new_kategori.drop_duplicates(subset=['kodekategori']).reset_index(drop=True)

In [ ]:
display(clean_kategori.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115
kodekategori,101.00,102.00,103.00,104.00,105.00,106.00,107.00,108.00,109.00,110.00,111.00,112.00,113.00,114.00,115.00,201.00,202.00,203.00,204.00,205.00,206.00,301.00,302.00,10101.00,10102.00,10103.00,10104.00,10105.00,10106.00,10107.00,10108.00,10109.00,10110.00,10111.00,10112.00,10113.00,10114.00,10115.00,10116.00,10117.00,10118.00,10119.00,10120.00,10121.00,10201.00,10202.00,10203.00,10204.00,10205.00,10206.00,10207.00,10208.00,10209.00,10210.00,10211.00,10212.00,10213.00,20101.00,20201.00,20301.00,20402.00,20501.00,20502.00,20503.00,20504.00,20601.00,30101.00,30201.00,30202.00,30203.00,30204.00,30301.00,30302.00,30303.00,30304.00,30305.00,30306.00,30307.00,30308.00,30309.00,30310.00,30311.00,30312.00,30401.00,30402.00,30403.00,30404.00,30405.00,30406.00,30407.00,40101.00,40103.00,40201.00,40202.00,40203.00,40301.00,40302.00,40303.00,40304.00,40305.00,40306.00,40307.00,40308.00,40309.00,40310.00,40311.00,40312.00,40401.00,40402.00,40403.00,40404.00,40405.00,40501.00,40601.00,20302.00,40502.00
kategori,['Baby Needs'],['Mens Care (2)'],['Healt & Beauty Care (3)'],['Kids Care (4)'],['Cleaning (5)'],['Homes (6)'],['Electronic & Tool kits (7)'],['Automotive (8)'],['Stationery (9)'],['Toys (10)'],['Sports (11)'],['Event Equipment (12)'],['Convection (13)'],['Shoes & Sandals (14)'],['Bag & Luggage (15)'],['Food (1)'],['Beverage (2)'],['Medicine (3)'],['Pets food (4)'],['Cigarette (5)'],['Fresh Food (6)'],['Others (1)'],['alisha collection'],['Makanan Bayi (Baby Food)'],['Bahan Kue (Baking Supplies)'],['Biskuit (Biscuit)'],['Roti dan Kue (Bread & Cake)'],['Sarapan Pagi (Breakfast)'],['Penganan Permen (Candy Confectionary)'],['Penganan Coklat (Chocolate Confectionary)'],['Makanan Mentah (Raw Food)'],['Ice'],['Makanan Masak (Cooked Food)'],['Bumbu Dapur (Seasoning)'],['Mie (Noodle)'],['Pasta'],['Makanan Ringan (Snack)'],['Beras (Rice)'],['Biji-bijian (Seeds)'],['Gula (Sugar)'],['Tepung (Flour)'],['Minyak Goreng (Cooking Oil)'],['Makanan Kaleng (Canned Food)'],['Makanan Hewan (Pet Food)'],['Susu (Milk)'],['Kopi (Coffe)'],['Teh (Tea)'],['Air (Water)'],['Minuman Buah (Juice)'],['Minuman Kesehatan (Healty Drink)'],['Minuman Berenergi (Energy Drink)'],['Minuman Isotonik (Isotonik Drink)'],['Minuman Penyegar (Refreshment)'],['Minuman Jeli (Jelly Drink)'],['Minuman Bersoda (Soft Drink)'],['Minuman Beralkohol (Alcoholic Drink)'],['Sirup (Syrup)'],['Buah (Fruit)'],['Sayur (Vegetable)'],['Daging Merah (Red Meat)'],['Air Tawar'],['Susu & Olahan (Dairy)'],['Harian (Daily)'],['Beku (Frozen)'],['Telur (Egg)'],['Produk Siap Makan (Ready To Eat)'],['Rokok (Cigarete)'],['Obat Dalam (Peroral)'],['Obat Luar (Topikal)'],['Vitamin & Suplemen (Vitamis & Supplements)'],['Alat Kesehatan (Health Appliance)'],['Perawatan Rambut (Hair Care)'],['Perawatan Wajah (Face Care)'],['Kosmetik (Cosmetic)'],['Tisu & Kapas (Tissues & Cotton)'],['Perawatan Gigi & Mulut (Oral Care)'],['Perawatan Tubuh (Body Care)'],['Perawatan Bayi (Baby Care)'],['Perawatan Anak (Kids Care)'],['Popok Bayi (Baby Diapers)'],['Pembalut Wanita (Sanitary Napkins)'],['Popok Dewasa (Adult Diapers)'],['Perawatan Pria (Mens Groming)'],['Perawatan Pakaian (Clothes Care)'],['Perawatan Dapur (Kitchen Care)'],['Perawatan Rumah (House Care)'],['Pengharum Rumah (Home Fragrances)'],['Pembasmi Serangga & Pengendali Hama (Insecti...,['Perawatan Tas & Sepatu Kulit (Leather Care)'],['Perawatan Otomotif (Automotive Care)'],['Mainan Lunak (Soft Toys)'],['Mainan Keras (Hard Toys)'],['Peralatan (Tools)'],['Perlengkapan (Supplies)'],['Fancies'],['Rumah Tangga (House Hold)'],['Dapur (Kitchen)'],['Kamar Mandi (Bath Room)'],['Kamar Tidur (Bed Room)'],['Peralatan & Perl

In [ ]:
# initiate the kategori list that will removed from dataset
kategori_remove_name = ["Fresh Food (6)", "Roti dan Kue (Bread & Cake)", "Sarapan Pagi (Breakfast)", "Makanan Mentah (Raw Food)", "Ice", "Makanan Masak (Cooked Food)", "Buah (Fruit)", "Sayur (Vegetable)", "Harian (Daily)", "Beku (Frozen)", "Telur (Egg)", "Produk Siap Makan (Ready To Eat)", "Daging Merah (Red Meat)", "Daging Putih (White Meat)"]
kategori_remove_list = ["206.00", "10104.00", "10105.00", "10108.00", "10109.00", "10110.00", "20101.00", "20201.00", "20502.00", "20503.00", "20504.00", "20601.00", "20301.00", "20302.00"]

In [ ]:
kategori_name_df = pd.DataFrame(kategori_remove_name)
# kategori_name_df.to_csv('kategori_delist.csv', index=False)

In [ ]:
# drop unnecessary column
df2020 = df2020.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2021 = df2021.drop(['notxn', 'nonota', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'iddistributor', 'idpromo', 'iddivisi', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)

In [ ]:
# concat 5 dataframe to single df
df = pd.concat([df2020, df2021]).reset_index(drop=True)

In [ ]:
# remove strange value in each rows from barcode column
df['barcode'] = df['barcode'].str.split(' ').str[0]

In [ ]:
df.head(10)

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori
0,2020-04-03 19:27:36.123,MASAKO PENYEDAP RS.SAPI/PAK,PAK,1.00,8992770033208,CU00000001,01011108,Bumbu Tabur,10111,Bumbu Dapur (Seasoning)
1,2020-10-21 16:47:57.980,RINSO 770GR ANTINODA,PCS,1.00,8999999401238,CU00000001,03040101,Detergent Bubuk,30401,Perawatan Pakaian (Clothes Care)
2,2020-04-06 17:46:52.900,CAREFREE HLT.FRESH SPR DRY 40S,PCS,1.00,8850007331803,CU00000001,03031001,Panty Liner,30310,Pembalut Wanita (Sanitary Napkins)
3,2020-07-31 19:15:58.547,ELLLIPS VIT HR NUTRI CLR BLS,PCS,2.00,8993417200038,CU00000001,03030105,Vitamin Rambut,30301,Perawatan Rambut (Hair Care)
4,2020-05-22 13:14:43.407,KUSUKA KRIPIK RUMPUT LAUT 60GR*30,PCS,1.00,653314504513,CU00000001,01011401,Snack Modern,10114,Makanan Ringan (Snack)
5,2020-09-03 20:51:42.103,HIT MAT 20S (18+3S),PCS,1.00,8992745120636,CU00000001,03040504,Pembasmi Serangga Elektrik,30405,Pembasmi Serangga & Pengendali Hama (Insectici...
6,2020-03-14 20:47:06.390,SARI ROTI SANDWICH MARGARIN GULA,PCS,1.00,8992907953171,CU00000001,01010406,Sandwich,10104,Roti dan Kue (Bread & Cake)
7,2020-09-21 17:08:36.647,BIORE BF COMFORT MILD SCRUB 100ML,PCS,1.00,8992727006514,CU00000001,03030605,Sabun Mandi Cair Aromatic,30306,Perawatan Tubuh (Body Care)
8,2020-02-02 20:47:39.680,SUNLIGHT EXTRA PURE REFF 24*175ML,PCS,1.00,8999999549510,CU00000001,03040201,Sabun Cair,30402,Perawatan Dapur (Kitchen Care)
9,2020-01-02 10:53:20.687,IZZI F MIST SECRET CRUSH 100ML,PCS,1.00,8992856899087,CU00000001,03031206,Cologne,30312,Perawatan Pria (Mens Groming)


In [ ]:
# drop unnesecary value from dataset 
df = df[df['kodecustomer'].isin(['CU00000001'])].reset_index(drop=True)
df = df[~df['barcode'].isin(['56101','56102','56103','56104'])].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('JASA PENDINGIN')].reset_index(drop=True)
df = df[~df['kategori'].str.contains('Toys', na=False)].reset_index(drop=True)
df = df[~df['namabarang'].str.contains('ZANDILAC')].reset_index(drop=True)

In [ ]:
# create new column named "month_year" to get number of month and year
df['month_year'] = df['tgl'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [ ]:
# transform the value to number of month from all rows
# 2020-2021
df['month_year'] = df['month_year'].map({'01-20':0,'02-20':1,'03-20':2,'04-20':3,'05-20':4,'06-20':5,'07-20':6,'08-20':7,'09-20':8,'10-20':9,'11-20':10,'12-20':11,
                                          '01-21':12,'02-21':13,'03-21':14,'04-21':15,'05-21':16,'06-21':17,'07-21':18,'08-21':19,'09-21':20,'10-21':21,'11-21':22,'12-21':23})

In [ ]:
# drop unnecessary kategori #tidak diremove
df = df[~df['kategori'].isin(kategori_remove_name)].reset_index(drop=True)

In [ ]:
display(df.info())
display(df.head())
# RangeIndex: 11533836 entries, 0 to 11533835 sebelum nambah daftar kategori

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3462361 entries, 0 to 3462360
Data columns (total 11 columns):
 #   Column        Dtype         
---  ------        -----         
 0   tgl           datetime64[ns]
 1   namabarang    object        
 2   satuan        object        
 3   qty           float64       
 4   barcode       object        
 5   kodecustomer  object        
 6   kodejenis     object        
 7   jenis         object        
 8   kodekategori  int64         
 9   kategori      object        
 10  month_year    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 290.6+ MB


None

,tgl,namabarang,satuan,qty,barcode,kodecustomer,kodejenis,jenis,kodekategori,kategori,month_year
0,2020-04-03 19:27:36.123,MASAKO PENYEDAP RS.SAPI/PAK,PAK,1.00,8992770033208,CU00000001,01011108,Bumbu Tabur,10111,Bumbu Dapur (Seasoning),3
1,2020-10-21 16:47:57.980,RINSO 770GR ANTINODA,PCS,1.00,8999999401238,CU00000001,03040101,Detergent Bubuk,30401,Perawatan Pakaian (Clothes Care),9
2,2020-04-06 17:46:52.900,CAREFREE HLT.FRESH SPR DRY 40S,PCS,1.00,8850007331803,CU00000001,03031001,Panty Liner,30310,Pembalut Wanita (Sanitary Napkins),3
3,2020-07-31 19:15:58.547,ELLLIPS VIT HR NUTRI CLR BLS,PCS,2.00,8993417200038,CU00000001,03030105,Vitamin Rambut,30301,Perawatan Rambut (Hair Care),6
4,2020-05-22 13:14:43.407,KUSUKA KRIPIK RUMPUT LAUT 60GR*30,PCS,1.00,653314504513,CU00000001,01011401,Snack Modern,10114,Makanan Ringan (Snack),4


In [ ]:
train_monthly = df[['tgl', 'month_year', 'kodejenis', 'barcode', 'qty']].copy()
train_monthly = train_monthly.sort_values('tgl').groupby(['month_year', 'kodejenis', 'barcode'], as_index=False)
train_monthly = train_monthly.agg({'qty':['sum']})
train_monthly.columns = ['month_year', 'kodejenis', 'barcode', 'qty_cnt']
train_monthly = train_monthly.query('qty_cnt > 0')

train_monthly['qty_cnt_month'] = train_monthly.sort_values('month_year').groupby(['barcode', 'kodejenis'])['qty_cnt'].shift(-1)

display(train_monthly.head())

,month_year,kodejenis,barcode,qty_cnt,qty_cnt_month
0,0,01010101,089686640007,30.00,24.00
1,0,01010101,089686640014,30.00,34.00
2,0,01010101,089686640045,12.00,12.00
3,0,01010101,089686646047,7.00,3.00
4,0,01010101,089686646061,5.00,6.00


In [ ]:
monthly_series = train_monthly.pivot_table(index=['barcode', 'kodejenis'], columns='month_year',values='qty_cnt', fill_value=0).reset_index()
monthly_series.head()

month_year,barcode,kodejenis,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,00100809,01011601,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00166454,01011109,0.77,1.51,1.39,0.30,0.40,0.27,0.35,0.27,0.14,0.17,0.13,0.07,0.60,0.11,0.27,0.00,0.22,0.17,0.17,0.33,0.51,0.29,0.05,0.36
2,00166461,01011109,7.14,9.01,29.51,9.92,5.89,6.63,2.98,4.06,4.44,4.54,1.56,2.67,3.03,4.35,3.56,2.23,3.61,2.73,5.10,6.68,5.16,6.00,4.11,2.94
3,00166478,01011109,1.06,0.64,2.01,1.11,0.86,1.34,0.67,0.15,0.42,0.41,0.05,0.58,0.70,0.17,0.77,0.21,0.34,0.33,0.00,0.58,1.12,0.23,0.81,0.16
4,00166485,01011109,1.70,3.10,4.53,2.33,1.93,0.17,0.43,0.17,0.54,0.37,0.11,0.08,0.30,0.78,0.63,0.20,0.57,0.35,0.49,0.21,0.95,0.42,0.28,0.91


In [ ]:
monthly_series_drop = monthly_series.drop(['kodejenis'], axis=1)

In [ ]:
summed_data = monthly_series_drop.groupby('barcode').agg('sum').reset_index(False)
summed_data.head()

month_year,barcode,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,00100809,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12,2.11,2.75,5.56,4.12,3.13,1.68,3.59
1,00166454,0.77,1.51,1.39,0.30,0.40,0.27,0.35,0.27,0.14,0.17,0.13,0.07,0.60,0.11,0.27,0.00,0.22,0.17,0.17,0.33,0.51,0.29,0.05,0.36
2,00166461,7.14,9.01,29.51,9.92,5.89,6.63,2.98,4.06,4.44,4.54,1.56,2.67,3.03,4.35,3.56,2.23,3.61,2.73,5.10,6.68,5.16,6.00,4.11,2.94
3,00166478,1.06,0.64,2.01,1.11,0.86,1.34,0.67,0.15,0.42,0.41,0.05,0.58,0.70,0.17,0.77,0.21,0.34,0.33,0.00,0.58,1.12,0.23,0.81,0.16
4,00166485,1.70,3.10,4.53,2.33,1.93,0.17,0.43,0.17,0.54,0.37,0.11,0.08,0.30,0.78,0.63,0.20,0.57,0.35,0.49,0.21,0.95,0.42,0.28,0.91


In [ ]:
first_month = 0
last_month = 23
serie_size = 12
data_series = []

for index, row in summed_data.iterrows():
    for month1 in range((last_month - (first_month + serie_size)) + 1):
        # serie = [row['barcode'], row['kodejenis']]
        serie = [row['barcode']]
        for month2 in range(serie_size + 1):
            serie.append(row[month1 + first_month + month2])
        data_series.append(serie)

columns = ['barcode']
[columns.append(i) for i in range(serie_size)]
columns.append('label')
data_series = pd.DataFrame(data_series, columns=columns)
data_series.head()

,barcode,0,1,2,3,4,5,6,7,8,9,10,11,label
0,00100809,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23
1,00100809,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75
2,00100809,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23
3,00100809,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37
4,00100809,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12


In [ ]:
data_series = data_series.drop(['barcode'], axis=1)
data_series.head()

,0,1,2,3,4,5,6,7,8,9,10,11,label
0,12.75,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23
1,10.78,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75
2,27.13,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23
3,32.93,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37
4,18.57,11.34,13.62,12.14,8.19,4.22,2.76,3.97,7.23,3.75,5.23,5.37,4.12


In [ ]:
outname = 'data_series_2020_2021_without_kategori.csv'
outdir = './data-train-unlimit'

if not os.path.exists(outdir):
    os.mkdir(outdir)

fullname = os.path.join(outdir, outname)    

data_series.to_csv(fullname)